In [1]:
import sys

import pandas as pd
import numpy as np
from openai import OpenAI

sys.path.append('../../inference/codes')
from api_secret import OPENAI_CLIENT_KEY_TMAXNLP

KeyboardInterrupt: 

In [ ]:
LANG_TABLE = {
    'en': 'English',
    'ko': '한국어',
    'ja': '日本語',
    'zh': '中文',
}

In [ ]:
def get_gpt_prompt(src_lang_code='en', tgt_lang_code='ko'):
    src_lang_full, tgt_lang_full = LANG_TABLE[src_lang_code], LANG_TABLE[tgt_lang_code]
    gpt_system_prompt = f"""
<instruction>
You are an assistant for making {src_lang_full}-{tgt_lang_full} text-pair data, for the another translation model.
You have to modify the given text-pair, by following the task guidelines below.
</instruction>
"""
    gpt_system_prompt = gpt_system_prompt.strip()
    return gpt_system_prompt

In [ ]:
class GptGenerator:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)

    def generate(self, prompt, gpt_version='gpt-4o-mini', seed=42, src_lang='en', tgt_lang='ko'):
        response = self.client.chat.completions.create(
            model=gpt_version,
            messages=[
                {"role": "system", "content": get_gpt_prompt(src_lang, tgt_lang)},
                {"role": "user", "content": prompt},
            ],
            temperature=1.0,
            seed=seed,
        )
        output = response.choices[0].message.content
        return output

In [ ]:
generator = GptGenerator(api_key=OPENAI_CLIENT_KEY_TMAXNLP)

In [ ]:
def get_user_prompt_linebreak(src_text, tgt_text, src_lang_code='en', tgt_lang_code='ko', n=1):
    src_lang_full, tgt_lang_full = LANG_TABLE[src_lang_code], LANG_TABLE[tgt_lang_code]

    task_instruction = f"""
Add {n} line-break(s) in each text in the pair.
The {n} line-break(s) can be located anywhere, even in the middle of the sentence.
But, the position of the line-breaks in two texts in the pair should be equivalent, regarding the semantic and syntactic structure of them.
Which means, the number of the line-breaks should be the same in the two texts, and the number of broken sentence should be {n+1}.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.
"""
    task_prompt = f"<task>{task_instruction}</task>"

    output_instruction = f"""
[SOURCE]\n{{src_text}}\n[/SOURCE]
[TARGET]\n{{tgt_text}}\n[/TARGET]
"""
    output_prompt = f"<output_template>{output_instruction}</output_template>"

    if n == 1:
        src_example = "안녕하세요. 제 이름은\nGPT-4o입니다."
        tgt_example = "Hello. My name is\nGPT-4o."
    elif n == 2:
        src_example = "안녕하세요.\n제 이름은\nGPT-4o입니다."
        tgt_example = "Hello.\nMy name is\nGPT-4o."
    elif n == 3:
        src_example = "안녕하세요.\n제\n이름은\nGPT-4o입니다."
        tgt_example = "Hello.\nMy\nname is\nGPT-4o."

    example_instruction = f"""
<source><한국어>\n안녕하세요. 제 이름은 GPT-4o입니다.\n</한국어></source>
<target><English>\nHello. My name is GPT-4o.\n</English></target>

[SOURCE]\n{src_example}\n[/SOURCE]
[TARGET]\n{tgt_example}\n[/TARGET]
"""
    example_prompt = f"<example>{example_instruction}</example>"

    source_prompt = f"<source><{src_lang_full}>\n{src_text}\n</{src_lang_full}></source>"
    target_prompt = f"<target><{tgt_lang_full}>\n{tgt_text}\n</{tgt_lang_full}></target>"

    user_prompt = f"{task_prompt}\n\n{output_prompt}\n\n{example_prompt}\n\n{source_prompt}\n{target_prompt}"
    user_prompt = user_prompt.strip()
    
    return user_prompt

In [ ]:
test_text_src = "지난 13일 '검은 금요일'이 지나간 주식 시장은 16일부터 다시 상승세를 보였다."
test_text_tgt = "The stock market, which passed the 'Black Friday' on the 13th, showed an upward trend again from the 16th."
prompt = get_user_prompt_linebreak(test_text_src, test_text_tgt, 'ko', 'en', 2)
print(f"{prompt}\n")

gpt_version = 'gpt-4o-mini'
generation = generator.generate(prompt, gpt_version=gpt_version, src_lang='ko', tgt_lang='en')
print(generation)

<task>
Add 2 line-break(s) in each text in the pair.
The 2 line-break(s) can be located anywhere, even in the middle of the sentence.
But, the position of the line-breaks in two texts in the pair should be equivalent, regarding the semantic and syntactic structure of them.
Which means, the number of the line-breaks should be the same in the two texts, and the number of broken sentence should be 3.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.
</task>

<output_template>
[SOURCE]
{src_text}
[/SOURCE]
[TARGET]
{tgt_text}
[/TARGET]
</output_template>

<example>
<source><한국어>
안녕하세요. 제 이름은 GPT-4o입니다.
</한국어></source>
<target><English>
Hello. My name is GPT-4o.
</English></target>

[SOURCE]
안녕하세요.
제 이름은
GPT-4o입니다.
[/SOURCE]
[TARGET]
Hello.
My name is
GPT-4o.
[/TARGET]
</example>

<source><한국어>
지난 13일 '검은 금요일'이 지나간 주식 시장은 16일부터 다시 상승세를 보였다.
</한국어></source>
<target><English>
The stock market, which p

In [ ]:
def get_user_prompt_propernoun(src_text, tgt_text, src_lang_code='en', tgt_lang_code='ko', trans=True):
    src_lang_full, tgt_lang_full = LANG_TABLE[src_lang_code], LANG_TABLE[tgt_lang_code]

    if trans:
        pn_instruction = f"translated into a target language ({tgt_lang_full})."
        input_example = "Eiffel Tower는 프랑스 파리 Champ de Mars에 있는 철제 격자 탑입니다."
        output_example = "에펠탑은 프랑스 파리의 샹드마르에 있는 철제 격자 탑입니다."
    else:
        pn_instruction = f"maintained as a source language ({src_lang_full})."
        input_example = "Eiffel Tower는 프랑스 Paris 샹드마르에 있는 철제 격자 탑입니다."
        output_example = "Eiffel Tower는 France Paris의 Champ de Mars에 있는 철제 격자 탑입니다."

    task_instruction = f"""
If there are proper nouns in the texts, those in the target text should be {pn_instruction}
If no change is needed, just leave the text as it is.
Do not change any other parts of the text, except for the proper nouns.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.
"""
    task_prompt = f"<task>{task_instruction}</task>"

    output_instruction = f"""
[TARGET]\n{{tgt_text}}\n[/TARGET]
"""
    output_prompt = f"<output_template>{output_instruction}</output_template>"

    example_instruction = f"""
<source><English>\nThe Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.\n</English></source>
<target><한국어>\n{input_example}\n</한국어></target>

[TARGET]\n{output_example}\n[/TARGET]
"""
    example_prompt = f"<example>{example_instruction}</example>"

    source_prompt = f"<source><{src_lang_full}>\n{src_text}\n</{src_lang_full}></source>"
    target_prompt = f"<target><{tgt_lang_full}>\n{tgt_text}\n</{tgt_lang_full}></target>"

    user_prompt = f"{task_prompt}\n\n{output_prompt}\n\n{example_prompt}\n\n{source_prompt}\n{target_prompt}"
    user_prompt = user_prompt.strip()
    
    return user_prompt

In [ ]:
test_text_src_lang = 'en'
test_text_tgt_lang = 'ko'
test_text_src = "Camp Nou is a football stadium in Barcelona, Spain."
test_text_tgt = "Camp Nou는 스페인 바르셀로나에 있는 축구 경기장입니다."
# test_text_src = "例えば、半導体産業の発展を触発した金属酸化膜半導体の電界効果トランジスター(MOSFET)を初めて発見した人が、まさに韓国人科学者であるカン・テウォン博士だ。"
# test_text_tgt = "예를 들어 반도체 산업의 발전을 촉발한 금속산화막 반도체 전계효과 트랜지스터(MOSFET)를 처음 발견한 이가 바로 한국인 과학자인 강대원 박사다."
# test_text_src = "业界排名第二的韩泰轮胎也因同种嫌疑接受了调查,并面临制裁。"
# test_text_tgt = "업계 2위인 한국타이어도 비슷한 혐의로 조사받고 제재를 앞두고 있다."

prompt = get_user_prompt_propernoun(test_text_src, test_text_tgt, test_text_src_lang, test_text_tgt_lang, trans=False)
print(f"{prompt}\n")

gpt_version = 'gpt-4o-mini'
generation = generator.generate(prompt, 
                                gpt_version=gpt_version, 
                                src_lang=test_text_src_lang, 
                                tgt_lang=test_text_tgt_lang)
print(generation)

<task>
If there are proper nouns in the texts, those in the target text should be maintained as a source language (English).
If no change is needed, just leave the text as it is.
Do not change any other parts of the text, except for the proper nouns.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.
</task>

<output_template>
[TARGET]
{tgt_text}
[/TARGET]
</output_template>

<example>
<source><English>
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
</English></source>
<target><한국어>
Eiffel Tower는 프랑스 Paris 샹드마르에 있는 철제 격자 탑입니다.
</한국어></target>

[TARGET]
Eiffel Tower는 France Paris의 Champ de Mars에 있는 철제 격자 탑입니다.
[/TARGET]
</example>

<source><English>
Camp Nou is a football stadium in Barcelona, Spain.
</English></source>
<target><한국어>
Camp Nou는 스페인 바르셀로나에 있는 축구 경기장입니다.
</한국어></target>

[TARGET]
Camp Nou는 스페인 Barcelona에 있는 축구 경기장입니다.
[/TARGET]


In [ ]:
def get_user_prompt_propernoun_contrastive(src_text, tgt_text, src_lang_code='en', tgt_lang_code='ko'):
    src_lang_full, tgt_lang_full = LANG_TABLE[src_lang_code], LANG_TABLE[tgt_lang_code]

    task_instruction = f"""
Two types of texts will be given, a source text in {src_lang_full} and a target text in {tgt_lang_full}.
You have to generate two types of modified target text:
(1) Maintained: Translated into {tgt_lang_full}, with {src_lang_full} proper nouns.
(2) Translated: Translated into {tgt_lang_full}, with {tgt_lang_full} proper nouns.

Do not change any other parts of the text, except for the proper nouns.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.

If there is no proper noun in the text, the output should be "N/A" within the template.
"""
    task_prompt = f"<task>{task_instruction}</task>"

    output_instruction = f"""
[PN_MAINTAINED]\n{{{tgt_lang_full} target text with {src_lang_full} proper nouns, or N/A}}\n[/PN_MAINTAINED]
[PN_TRANSLATED]\n{{{tgt_lang_full} target text with {tgt_lang_full} proper nouns, or N/A}}\n[/PN_TRANSLATED]
"""
    output_prompt = f"<output_template>{output_instruction}</output_template>"

    example_instruction = f"""
<source><English>\nThe Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.\n</English></source>
<target><한국어>\n에펠탑은 프랑스 파리 Champ de Mars에 있는 철제 격자 탑입니다.\n</한국어></target>

[PN_MAINTAINED]\nEiffel Tower는 France Paris의 Champ de Mars에 있는 철제 격자 탑입니다.\n[/PN_MAINTAINED]
[PN_TRANSLATED]\n에펠탑은 프랑스 파리의 샹드마르에 있는 철제 격자 탑입니다.\n[/PN_TRANSLATED]
"""
    example_prompt = f"<example>{example_instruction}</example>"

    source_prompt = f"<source><{src_lang_full}>\n{src_text}\n</{src_lang_full}></source>"
    target_prompt = f"<target><{tgt_lang_full}>\n{tgt_text}\n</{tgt_lang_full}></target>"

    user_prompt = f"{task_prompt}\n\n{output_prompt}\n\n{example_prompt}\n\n{source_prompt}\n{target_prompt}"
    user_prompt = user_prompt.strip()
    
    return user_prompt

In [ ]:
test_text_src_lang = 'en'
test_text_tgt_lang = 'ko'
# test_text_src = "Camp Nou is a football stadium in Barcelona, Spain."
# test_text_tgt = "Camp Nou는 스페인 바르셀로나에 있는 축구 경기장입니다."
# test_text_src = "例えば、半導体産業の発展を触発した金属酸化膜半導体の電界効果トランジスター(MOSFET)を初めて発見した人が、まさに韓国人科学者であるカン・テウォン博士だ。"
# test_text_tgt = "예를 들어 반도체 산업의 발전을 촉발한 금속산화막 반도체 전계효과 트랜지스터(MOSFET)를 처음 발견한 이가 바로 한국인 과학자인 강대원 박사다."
# test_text_src = "业界排名第二的韩泰轮胎也因同种嫌疑接受了调查,并面临制裁。"
# test_text_tgt = "업계 2위인 한국타이어도 비슷한 혐의로 조사받고 제재를 앞두고 있다."
test_text_src = "On behalf of Cafe Boulud NYC at CITY HARVEST BID 2019, a famous culinary charity event held in October last year, he presented Charcuterie, a traditional French dish."
test_text_tgt = "지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 2019'에 Cafe Boulud NYC를 대표해 나가 프랑스 대표 전통요리인 샤퀴테리(Charcuterie) 요리를 선보였다."
# test_text_src = "I was wondering if you could help me out with this ukulele."
# test_text_tgt = "이 우쿨렐레 관련해서 도와주세요."

prompt = get_user_prompt_propernoun_contrastive(test_text_src, test_text_tgt, test_text_src_lang, test_text_tgt_lang)
print(f"{prompt}\n")

gpt_version = 'gpt-4o-mini'
generation = generator.generate(prompt, 
                                gpt_version=gpt_version, 
                                src_lang=test_text_src_lang, 
                                tgt_lang=test_text_tgt_lang)
print(generation)

<task>
Two types of texts will be given, a source text in English and a target text in 한국어.
You have to generate two types of modified target text:
(1) Maintained: Translated into 한국어, with English proper nouns.
(2) Translated: Translated into 한국어, with 한국어 proper nouns.

Do not change any other parts of the text, except for the proper nouns.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.

If there is no proper noun in the text, the output should be "N/A" within the template.
</task>

<output_template>
[PN_MAINTAINED]
{한국어 target text with English proper nouns, or N/A}
[/PN_MAINTAINED]
[PN_TRANSLATED]
{한국어 target text with 한국어 proper nouns, or N/A}
[/PN_TRANSLATED]
</output_template>

<example>
<source><English>
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
</English></source>
<target><한국어>
에펠탑은 프랑스 파리 Champ de Mars에 있는 철제 격자 탑입니다.
</한국어></target>

[

In [ ]:
def get_user_prompt_style(src_text, tgt_text, src_lang_code='en', tgt_lang_code='ko', formal=True):
    src_lang_full, tgt_lang_full = LANG_TABLE[src_lang_code], LANG_TABLE[tgt_lang_code]

    if formal:
        style = 'formal and polite'
        input_example_src = "당국은 이번 주말에 코로나 백신 접종을 받으러 오는 사람들에게 무료로 마스크를 제공할 예정이라고 밝혔다."
        input_example_tgt = "The government said they’re gonna hand out free masks to anyone who shows up for their COVID shot this weekend."
        output_example = "The authorities have announced their plan to distribute free masks to individuals who arrive for their COVID-19 vaccination this weekend."
    else:
        style = 'informal and casual'
        input_example_src = "당국은 이번 주말에 코로나 백신 접종을 받으러 오는 사람들에게 무료로 마스크를 제공할 예정이라고 밝혔다."
        input_example_tgt = "The authorities have announced their plan to distribute free masks to individuals who arrive for their COVID-19 vaccination this weekend."
        output_example = "The government said they’re gonna hand out free masks to anyone who shows up for their COVID shot this weekend."

    task_instruction = f"""
Change the style of the target text to more {style} style.
The structure and vocabulary of the target text can change significantly based on the specified style.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.
"""
    task_prompt = f"<task>{task_instruction}</task>"

    output_instruction = f"""
[TARGET]\n{{tgt_text}}\n[/TARGET]
"""
    output_prompt = f"<output_template>{output_instruction}</output_template>"

    example_instruction = f"""
<source><한국어>\n{input_example_src}\n</한국어></source>
<target><English>\n{input_example_tgt}\n</English></target>

[TARGET]\n{output_example}\n[/TARGET]
"""
    example_prompt = f"<example>{example_instruction}</example>"

    source_prompt = f"<source><{src_lang_full}>\n{src_text}\n</{src_lang_full}></source>"
    target_prompt = f"<target><{tgt_lang_full}>\n{tgt_text}\n</{tgt_lang_full}></target>"

    user_prompt = f"{task_prompt}\n\n{output_prompt}\n\n{example_prompt}\n\n{source_prompt}\n{target_prompt}"
    user_prompt = user_prompt.strip()

    return user_prompt

In [ ]:
test_text_src_lang = 'en'
test_text_tgt_lang = 'ko'
test_text_src = "Thanks a lot for sticking with us and joining in on our program!"
test_text_tgt = "항상 우리 프로그램에 관심 가져주고 참여해줘서 정말 고마워!"

prompt = get_user_prompt_style(test_text_src, test_text_tgt, test_text_src_lang, test_text_tgt_lang, formal=True)
print(f"{prompt}\n")

gpt_version = 'gpt-4o-mini'
generation = generator.generate(prompt, 
                                gpt_version=gpt_version, 
                                src_lang=test_text_src_lang, 
                                tgt_lang=test_text_tgt_lang)
print(generation)

<task>
Change the style of the target text to more formal and polite style.
The structure and vocabulary of the target text can change significantly based on the specified style.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.
</task>

<output_template>
[TARGET]
{tgt_text}
[/TARGET]
</output_template>

<example>
<source><한국어>
당국은 이번 주말에 코로나 백신 접종을 받으러 오는 사람들에게 무료로 마스크를 제공할 예정이라고 밝혔다.
</한국어></source>
<target><English>
The government said they’re gonna hand out free masks to anyone who shows up for their COVID shot this weekend.
</English></target>

[TARGET]
The authorities have announced their plan to distribute free masks to individuals who arrive for their COVID-19 vaccination this weekend.
[/TARGET]
</example>

<source><English>
Thanks a lot for sticking with us and joining in on our program!
</English></source>
<target><한국어>
항상 우리 프로그램에 관심 가져주고 참여해줘서 정말 고마워!
</한국어></target>

[TARGET]
귀하께서 

In [ ]:
def get_user_prompt_style_contrastive(src_text, tgt_text, src_lang_code='en', tgt_lang_code='ko'):
    src_lang_full, tgt_lang_full = LANG_TABLE[src_lang_code], LANG_TABLE[tgt_lang_code]

    task_instruction = f"""
Two types of texts will be given, a source text in {src_lang_full} and a target text in {tgt_lang_full}.
You have to generate two types of outputs:
(1) Translated into a target language, in formal and polite (written-alike) style.
(2) Translated into a target language, in informal and casual (colloquial-alike) style.
The structure and vocabulary of the target text can change significantly based on the specified style.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.
"""
    task_prompt = f"<task>{task_instruction}</task>"

    output_instruction = f"""
[FORMAL]\n{{tgt text in formal style}}\n[/FORMAL]
[INFORMAL]\n{{tgt text in informal style}}\n[/INFORMAL]
"""
    output_prompt = f"<output_template>{output_instruction}</output_template>"

    example_instruction = f"""
<source><English>\nThe authorities have announced their plan to distribute free masks to individuals who arrive for their COVID-19 vaccination this weekend.\n</English></source>
<target><한국어>\n당국은 이번 주말에 코로나 백신 접종을 받으러 오는 사람들에게 무료로 마스크를 제공할 예정이라고 밝혔다.\n</한국어></target>

[FORMAL]\n정부는 금주 주말 코로나 백신 접종을 위해 방문하는 이들에게 무료 마스크를 제공할 예정이라 공지했다.\n[/FORMAL]
[INFORMAL]\n정부에서 이번 주말에 코로나 백신 맞으러 오는 사람들한테 공짜로 마스크 나눠준다고 했어요.\n[/FORMAL]
"""
    example_prompt = f"<example>{example_instruction}</example>"

    source_prompt = f"<source><{src_lang_full}>\n{src_text}\n</{src_lang_full}></source>"
    target_prompt = f"<target><{tgt_lang_full}>\n{tgt_text}\n</{tgt_lang_full}></target>"

    user_prompt = f"{task_prompt}\n\n{output_prompt}\n\n{example_prompt}\n\n{source_prompt}\n{target_prompt}"
    user_prompt = user_prompt.strip()

    return user_prompt

In [ ]:
test_text_src_lang = 'en'
test_text_tgt_lang = 'ko'
test_text_src = "Thanks a lot for sticking with us and joining in on our program!"
test_text_tgt = "늘상 우리 프로그램에 관심 가져주고 참여해줘서 정말 감사합니다."

prompt = get_user_prompt_style_contrastive(test_text_src, test_text_tgt, test_text_src_lang, test_text_tgt_lang)
print(f"{prompt}\n")

gpt_version = 'gpt-4o-mini'
generation = generator.generate(prompt, 
                                gpt_version=gpt_version, 
                                src_lang=test_text_src_lang, 
                                tgt_lang=test_text_tgt_lang)
print(generation)

<task>
Two types of texts will be given, a source text in English and a target text in 한국어.
You have to generate two types of outputs:
(1) Translated into a target language, in formal and polite (written-alike) style.
(2) Translated into a target language, in informal and casual (colloquial-alike) style.
The structure and vocabulary of the target text can change significantly based on the specified style.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.
</task>

<output_template>
[FORMAL]
{tgt text in formal style}
[/FORMAL]
[INFORMAL]
{tgt text in informal style}
[/INFORMAL]
</output_template>

<example>
<source><English>
The authorities have announced their plan to distribute free masks to individuals who arrive for their COVID-19 vaccination this weekend.
</English></source>
<target><한국어>
당국은 이번 주말에 코로나 백신 접종을 받으러 오는 사람들에게 무료로 마스크를 제공할 예정이라고 밝혔다.
</한국어></target>

[FORMAL]
정부는 금주 주말 코로나 백신 접

In [ ]:
not_prime = pd.read_csv('../../inference/results/prime-cleansing/sft/mmt_not_prime_train.csv')
not_prime.head()

,id,src,src_xcomet,tgt,tgt_xcomet,data_source,direction
0,00111-enko-0000002,"In particular, typhoons and inclement weather ...",NaN,"특히 태풍과 궂은 날씨로 일부 가을 축제가 이미 차질을 빚은데다, 지역 경제가 어려...",NaN,aihub-111,en-ko
1,00111-enko-0000003,"Currently, Korea Battery Fund No. 1 and Kolink...",NaN,현재 더블유에프엠은 한국배터리펀드1호와 코링크PE가 합산 12%의 지분을 보유한 대...,NaN,aihub-111,en-ko
2,00111-enko-0000004,"Four portraits of Lee Hang-ro, Yoo Jung-gyo, Y...",NaN,이항로·유중교·유인석·이소응 네 분의 영정은 나라를 위해 목숨을 바친 수많은 의병들...,NaN,aihub-111,en-ko
3,00111-enko-0000005,"If the veterans along with Kim Ha-sung, Lee Ju...",NaN,"베테랑과 김하성, 이정후, 김혜성 등이 시너지 효과를 일으키면 넥센은 어느 팀과 비...",NaN,aihub-111,en-ko
4,00111-enko-0000006,"Yerim village and Seopyeon village, the main v...",NaN,예림리의 본동 마을인 예림 마을과 서편 마을은 대부분 공단 지구에 편입되어 옛 흔적...,NaN,aihub-111,en-ko


In [ ]:
data_num = 2
print(not_prime[not_prime['direction']=='zh-ko']['src'].iloc[data_num])
print(not_prime[not_prime['direction']=='zh-ko']['tgt'].iloc[data_num])

业界排名第二的韩泰轮胎也因同种嫌疑接受了调查,并面临制裁。
업계 2위인 한국타이어도 비슷한 혐의로 조사받고 제재를 앞두고 있다.


In [ ]:
data_sources = not_prime['data_source'].unique().tolist()
lang_pairs = not_prime['direction'].unique().tolist()

In [ ]:
dpo_substitutes = pd.DataFrame()
for source in data_sources:
    for lang_pair in lang_pairs:
        target_data = not_prime[(not_prime['data_source'] == source) & (not_prime['direction'] == lang_pair)]
        if target_data.empty:
            continue
        if len(target_data) >= 10000:
            target_sample = target_data.sample(n=10000, random_state=42)
        else:
            target_sample = target_data
        dpo_substitutes = pd.concat([dpo_substitutes, target_sample])

print(len(dpo_substitutes))

266525


In [ ]:
dpo_substitutes = dpo_substitutes.reset_index(drop=True)
dpo_substitutes.head()

,id,src,src_xcomet,tgt,tgt_xcomet,data_source,direction
0,00111-enko-0108202,"Bridges of river tributaries and small rivers,...",NaN,"하천 지류나 소하천의 교량은 이섭교와 세병교 등과 같이 조선 시대부터 조성되었으며,...",NaN,aihub-111,en-ko
1,00111-enko-0091695,"Ucomm Technology Co., Ltd., a golf IT company,...",NaN,골프 IT 기업인 보이스캐디 ㈜유컴테크놀러지가 지난 8월 29일 센추리 21CC에서...,NaN,aihub-111,en-ko
2,00111-enko-0028507,It is about how Japanese TVs using vacuum tube...,NaN,과거 진공관을 사용한 일본 TV는 세계를 제패했던 1등 제품이었으나 기술 대세가 L...,NaN,aihub-111,en-ko
3,00111-enko-0097052,"Dr. Mark Gorillic, a pediatric rheumatoid and ...",NaN,컬럼비아대병원의 소아 류마티스·면역의학과 전문의 마크 고릴릭 박사는 3명의 소아 환...,NaN,aihub-111,en-ko
4,00111-enko-0013462,"Of the 100 products, 87 exceeded the standard ...",NaN,100개 제품 중 87개 제품은 붕소가 기준치를 초과하였고 17개 제품은 붕소 외 ...,NaN,aihub-111,en-ko


In [ ]:
display(dpo_substitutes['data_source'].value_counts())
display(dpo_substitutes['direction'].value_counts())

data_source
aihub-71263    20000
aihub-71593    20000
aihub-546      20000
aihub-71493    20000
aihub-71496    19775
aihub-71498    19107
aihub-563      10000
aihub-125      10000
aihub-124      10000
aihub-111      10000
aihub-126      10000
aihub-127      10000
aihub-71265    10000
aihub-71591    10000
aihub-71266    10000
aihub-71382    10000
aihub-71524    10000
aihub-129      10000
aihub-128      10000
aihub-71262    10000
aihub-71411     7643
Name: count, dtype: int64

direction
ja-ko    94526
zh-ko    91999
en-ko    80000
Name: count, dtype: int64

In [ ]:
test_num = 0
test_lang_pair = dpo_substitutes.loc[test_num, 'direction']
test_src_lang, test_tgt_lang = test_lang_pair.split('-')
test_src = dpo_substitutes.loc[test_num, 'src']
test_tgt = dpo_substitutes.loc[test_num, 'tgt']

print(f"Source lang: {test_src_lang} / Target lang: {test_tgt_lang}\n")
print(f"[Source]\n{test_src}\n")
print(f"[Target]\n{test_tgt}")

Source lang: en / Target lang: ko

[Source]
Bridges of river tributaries and small rivers, like Lee Seop Bridge and Sebyung Bridge, have been built since the Joseon Dynasty, and then have increased as roads were built.

[Target]
하천 지류나 소하천의 교량은 이섭교와 세병교 등과 같이 조선 시대부터 조성되었으며, 이후 도로가 개설되면서 늘어나게 된다.


In [ ]:
import pandas as pd

In [ ]:
req_lb = pd.read_json('./gpt_dpo_linebreak_request.jsonl', lines=True)
res_lb = pd.read_json('./gpt_dpo_linebreak_response.jsonl', lines=True)
req_pn = pd.read_json('./gpt_dpo_propernoun_request.jsonl', lines=True)
res_pn = pd.read_json('./gpt_dpo_propernoun_response.jsonl', lines=True)
req_st = pd.read_json('./gpt_dpo_style_request.jsonl', lines=True)
res_st = pd.read_json('./gpt_dpo_style_response.jsonl', lines=True)

In [ ]:
import re

def extract_tagged_content(text, start_tag, end_tag, group=0):
    pattern = fr'{re.escape(start_tag)}(.*?){re.escape(end_tag)}'
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    if not matches:
        return None
    return matches[group]


def extract_first_and_last_tags(xml_text):
    first_tag_match = re.findall(r'<\s*([^/][^>\s]*)[^>]*>', xml_text)
    if first_tag_match:
        tag_src = first_tag_match[0]
    else:
        tag_src = None

    last_tag_match = re.findall(r'<\s*/\s*([^>\s]+)[^>]*>', xml_text)
    if last_tag_match:
        tag_tgt = last_tag_match[-1]
    else:
        tag_tgt = None

    return tag_src, tag_tgt


def check_req_and_res_whole_content(req_df, res_df, data_num=0):
    request = req_df['body'][data_num]['messages'][1]['content'].strip()
    response = res_df['response'][data_num]['body']['choices'][0]['message']['content'].strip()

    print(f"[Request]\n{request}\n")
    print(f"[Response]\n{response}")


def check_req_and_res_for_linebreak(req_df, res_df, data_num=0):
    request = req_df['body'][data_num]['messages'][1]['content'].strip()
    request_src_tgt = extract_tagged_content(request, '<source>', '</target>', 1).strip()
    tag_src, tag_tgt = extract_first_and_last_tags(request_src_tgt)
    request_src = extract_tagged_content(request_src_tgt, f'<{tag_src}>', f'</{tag_src}>', 0).strip()
    request_tgt = extract_tagged_content(request_src_tgt, f'<{tag_tgt}>', f'</{tag_tgt}>', 0).strip()

    response = res_df['response'][data_num]['body']['choices'][0]['message']['content'].strip()
    response_src = extract_tagged_content(response, '[SOURCE]', '[/SOURCE]', 0).strip()
    response_tgt = extract_tagged_content(response, '[TARGET]', '[/TARGET]', 0).strip()

    print(f"[Request]\n{request_src}\n{request_tgt}\n")
    print(f"[Response]\n{response_src}\n{response_tgt}")


def check_req_and_res_for_propernoun(req_df, res_df, data_num=0):
    request = req_df['body'][data_num]['messages'][1]['content'].strip()
    request_src_tgt = extract_tagged_content(request, '<source>', '</target>', 1).strip()
    tag_src, tag_tgt = extract_first_and_last_tags(request_src_tgt)
    request_src = extract_tagged_content(request_src_tgt, f'<{tag_src}>', f'</{tag_src}>', 0).strip()
    request_tgt = extract_tagged_content(request_src_tgt, f'<{tag_tgt}>', f'</{tag_tgt}>', 0).strip()

    response = res_df['response'][data_num]['body']['choices'][0]['message']['content'].strip()
    response_trans = extract_tagged_content(response, '[PN_MAINTAINED]', '[/PN_MAINTAINED]', 0).strip()
    response_untrans = extract_tagged_content(response, '[PN_TRANSLATED]', '[/PN_TRANSLATED]', 0).strip()

    print(f"[Request]\n{request_src}\n{request_tgt}\n")
    print(f"[Response]\n{response_trans}\n{response_untrans}")


def check_req_and_res_for_style(req_df, res_df, data_num=0):
    request = req_df['body'][data_num]['messages'][1]['content'].strip()
    request_src_tgt = extract_tagged_content(request, '<source>', '</target>', 1).strip()
    tag_src, tag_tgt = extract_first_and_last_tags(request_src_tgt)
    request_src = extract_tagged_content(request_src_tgt, f'<{tag_src}>', f'</{tag_src}>', 0).strip()
    request_tgt = extract_tagged_content(request_src_tgt, f'<{tag_tgt}>', f'</{tag_tgt}>', 0).strip()

    response = res_df['response'][data_num]['body']['choices'][0]['message']['content'].strip()
    response_trans = extract_tagged_content(response, '[FORMAL]', '[/FORMAL]', 0).strip()
    response_untrans = extract_tagged_content(response, '[INFORMAL]', '[/INFORMAL]', 0).strip()

    print(f"[Request]\n{request_src}\n{request_tgt}\n")
    print(f"[Response]\n{response_trans}\n{response_untrans}")

In [ ]:
check_req_and_res_for_linebreak(req_lb, res_lb, data_num=0)

[Request]
Blockchain is the transaction information that securely records all FFFs as a database book.
블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.

[Response]
Blockchain is the transaction information that securely 
records all FFFs as a database book.
블록체인은 하나의 데이터베이스 장부로서 모든 
FFF를 안전하게 기록한 거래정보를 말한다.


In [ ]:
check_req_and_res_for_propernoun(req_pn, res_pn, data_num=2)

[Request]
The meeting will also be attended by first ladies of African leaders such as Angola, Kenya, Nigeria, Tanzania and Zambia, as well as AAA2, wife of British Prime Minister AAA1, and AAA4, wife of California governor AAA3.
영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 영부인들과 영국 AAA1 총리의 부인 AAA2, 캘리포니아주지사인 AAA3의 부인 AAA4 등도 참석한다.

[Response]
영부인 회동에는 Angola, Kenya, Nigeria, Tanzania, Zambia 등 아프리카 정상의 영부인들과 British 총리 AAA1의 부인 AAA2, California 주지사 AAA3의 부인 AAA4 등도 참석한다.
영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 영부인들과 영국 총리 AAA1의 부인 AAA2, 캘리포니아 주지사 AAA3의 부인 AAA4 등도 참석한다.


In [ ]:
check_req_and_res_for_style(req_st, res_st, data_num=13)

[Request]
To provide such a structure in a short arc discharge lamp comprising a luminous tube, sealing tubes connected to both ends of the luminous tube, a pair of electrodes arranged oppositely in the luminous tube, and an electrode core wire supporting the electrode that even if an electrode core wire holding body is arranged in the sealing tube, the glass material fused when sealing the sealing tube does not hang on the front of the electrode core wire and fuse thereto, and occurrence of cracking from that part is prevented. A cushioning foil wound around the outer periphery of an electrode core wire holding body is projecting farther to the luminous tube side than the front end face of the electrode core wire holding body.
발광관과 상기 발광관의 양단에 접속된 밀봉관과 상기 발광관 내에 대향 배치된 한 쌍의 전극과 상기 전극을 지지하는 전극심선을 구비해서 이루어지는 쇼트 아크형 방전 램프에 있어서 밀봉관 내에 금속제의 전극심선 지지체를 배치해도, 밀봉관의 밀봉 시에 용융한 유리 재료가 전극심선 지지체의 전면에 처져 융착하는 일이 없도록 하여 그 부분으로부터의 균열 발생을 방지한 구조를 제공하는 것이다. 상기 전극심선 지지체의 외측 둘레에 감겨진 완충박이 상기 전극심선 지지체 전단면보다

In [ ]:
display(req_lb.head(1))
display(res_lb.head(1))

,custom_id,method,url,body
0,linebreak-enko-1,POST,/v1/chat/completions,"{'model': 'gpt-4o-mini', 'messages': [{'role':..."


,id,custom_id,response,error
0,batch_req_5h3OmzjGpLG1Pt48n097TKfa,linebreak-enko-1,"{'status_code': 200, 'request_id': '666abcdc66...",NaN


In [ ]:
def extract_req_and_res(row_req, row_res, tag_src, tag_tgt):
    request = row_req['body']['messages'][1]['content'].strip()
    request_src_tgt = extract_tagged_content(request, '<source>', '</target>', 1).strip()
    tag_src_lang, tag_tgt_lang = extract_first_and_last_tags(request_src_tgt)
    request_src = extract_tagged_content(request_src_tgt, f'<{tag_src_lang}>', f'</{tag_src_lang}>', 0).strip()
    request_tgt = extract_tagged_content(request_src_tgt, f'<{tag_tgt_lang}>', f'</{tag_tgt_lang}>', 0).strip()

    response = row_res['response']['body']['choices'][0]['message']['content'].strip()
    try:
        response_src = extract_tagged_content(response, f'[{tag_src}]', f'[/{tag_src}]', 0).strip()
    except:
        response_src = None
    try:
        response_tgt = extract_tagged_content(response, f'[{tag_tgt}]', f'[/{tag_tgt}]', 0).strip()
    except:
        response_tgt = None

    return request_src, request_tgt, response_src, response_tgt

In [ ]:
from tqdm import tqdm
from collections import defaultdict

def merge_req_and_res_for_linebreak(req_df, res_df):
    merged_df = defaultdict(list)

    error_ids = []
    for (_, row_req), (_, row_res) in tqdm(zip(req_df.iterrows(), res_df.iterrows()), total=len(req_df), desc='Merging Linbreak DF'):
        id = row_req['custom_id']
        id = id.replace('modification-', '')
        lang_pair = id.split('-')[1]
        direction = lang_pair[:2] + '-' + lang_pair[2:]

        request_src, request_tgt, response_src, response_tgt = extract_req_and_res(row_req, row_res, 'SOURCE', 'TARGET')
        if response_src is None or response_tgt is None:
            error_ids.append(id)

        merged_df['id'].append(id)
        merged_df['direction'].append(direction)
        merged_df['in_src'].append(request_src)
        merged_df['in_tgt'].append(request_tgt)
        merged_df['out_src'].append(response_src)
        merged_df['out_tgt'].append(response_tgt)

    merged_df = pd.DataFrame(merged_df)

    return merged_df, error_ids

In [ ]:
merged_lb, error_ids_lb = merge_req_and_res_for_linebreak(req_lb, res_lb)
display(merged_lb.head())
print(error_ids_lb)
print(len(merged_lb))

Merging Linbreak DF: 100%|██████████| 3900/3900 [00:00<00:00, 8769.07it/s]


,id,direction,in_src,in_tgt,out_src,out_tgt
0,linebreak-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...
1,linebreak-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다."
2,linebreak-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상..."
3,linebreak-enko-4,en-ko,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...
4,linebreak-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...


[]
3900


In [ ]:
req_lb = pd.read_json('./gpt_dpo_linebreak_request.jsonl', lines=True)
res_lb = pd.read_json('./gpt_dpo_linebreak_modification_response.jsonl', lines=True)

In [ ]:
merged_lb, error_ids_lb = merge_req_and_res_for_linebreak(req_lb, res_lb)
display(merged_lb.head())
print(len(error_ids_lb), error_ids_lb)
print(len(merged_lb))

Merging Linbreak DF: 100%|█████████▉| 3899/3900 [00:00<00:00, 8719.63it/s]


,id,direction,in_src,in_tgt,out_src,out_tgt
0,linebreak-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...
1,linebreak-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다."
2,linebreak-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
3,linebreak-enko-4,en-ko,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...
4,linebreak-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...


0 []
3899


In [ ]:
merged_lb.to_csv('./gpt_dpo_linebreak_modification_merged.csv', index=False)

In [ ]:
def merge_req_and_res_for_propernoun(req_df, res_df):
    merged_df = defaultdict(list)

    error_ids = []
    for (_, row_req), (_, row_res) in tqdm(zip(req_df.iterrows(), res_df.iterrows()), total=len(req_df), desc='Merging Propernoun DF'):
        id = row_req['custom_id']
        lang_pair = id.split('-')[1]
        direction = lang_pair[:2] + '-' + lang_pair[2:]

        request_src, request_tgt, response_src, response_tgt = extract_req_and_res(row_req, row_res, 'PN_MAINTAINED', 'PN_TRANSLATED')
        if response_src is None or response_tgt is None:
            error_ids.append(id)

        merged_df['id'].append(id)
        merged_df['direction'].append(direction)
        merged_df['in_src'].append(request_src)
        merged_df['in_tgt'].append(request_tgt)
        merged_df['out_src'].append(response_src)
        merged_df['out_tgt'].append(response_tgt)

    merged_df = pd.DataFrame(merged_df)

    return merged_df, error_ids

In [ ]:
merged_pn, error_ids_pn = merge_req_and_res_for_propernoun(req_pn, res_pn)
display(merged_pn.head())
print(error_ids_pn)
print(len(merged_pn))

Merging Propernoun DF: 100%|██████████| 13200/13200 [00:01<00:00, 8730.17it/s]


,id,direction,in_src,in_tgt,out_src,out_tgt
0,propernoun-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,N/A
1,propernoun-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.",N/A
2,propernoun-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...","영부인 회동에는 Angola, Kenya, Nigeria, Tanzania, Zam...","영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
3,propernoun-enko-4,en-ko,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,N/A,N/A
4,propernoun-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 '시티 하베스트 비드 2019'에 카페...


['propernoun-koja-7792']
13200


In [ ]:
def merge_req_and_res_for_style(req_df, res_df):
    merged_df = defaultdict(list)

    error_ids = []
    for (_, row_req), (_, row_res) in tqdm(zip(req_df.iterrows(), res_df.iterrows()), total=len(req_df), desc='Merging Style DF'):
        id = row_req['custom_id']
        lang_pair = id.split('-')[1]
        direction = lang_pair[:2] + '-' + lang_pair[2:]

        request_src, request_tgt, response_src, response_tgt = extract_req_and_res(row_req, row_res, 'FORMAL', 'INFORMAL')
        if response_src is None or response_tgt is None:
            error_ids.append(id)

        merged_df['id'].append(id)
        merged_df['direction'].append(direction)
        merged_df['in_src'].append(request_src)
        merged_df['in_tgt'].append(request_tgt)
        merged_df['out_src'].append(response_src)
        merged_df['out_tgt'].append(response_tgt)

    merged_df = pd.DataFrame(merged_df)

    return merged_df, error_ids

In [ ]:
merged_st, error_ids_st = merge_req_and_res_for_style(req_st, res_st)
display(merged_st.head())
print(error_ids_st)
print(len(merged_st))

Merging Style DF: 100%|██████████| 7800/7800 [00:00<00:00, 8955.25it/s]


,id,direction,in_src,in_tgt,out_src,out_tgt
0,style-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,블록체인이란 모든 FFF를 안전하게 기록한 거래 정보를 하나의 데이터베이스 장부로 ...,블록체인이란 모든 FFF를 안전하게 기록한 거래 정보 같은 데이터베이스 장부야.
1,style-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.",그러나 신장 생검이 pSS 환자의 최소 변화 질환에 대한 진단을 확인하였습니다.,근데 신장 생검으로 pSS 환자의 최소 변화 질환 진단이 확실해졌어요.
2,style-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...","회의에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 영부인들...","이번 회의에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아의 영부인들과 영국 총리..."
3,style-enko-4,en-ko,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,커프형 전극을 통한 신경 신호 수집은 FNS 시스템에서 필요한 정보 제공 및 피드백...,커프형 전극으로 신경 신호를 수집하는 건 FNS 시스템에서 필요한 정보 전달과 피드...
4,style-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,그는 지난해 10월에 열린 유명한 요리 자선 행사인 'CITY HARVEST BID...,그는 작년 10월에 열린 유명한 요리 자선 행사 'CITY HARVEST BID 2...


['style-enko-18', 'style-enko-133', 'style-enko-954', 'style-enko-1117', 'style-koen-1685', 'style-koen-1702', 'style-jako-2693', 'style-jako-2764', 'style-jako-2983', 'style-jako-3163', 'style-jako-3245', 'style-jako-3249', 'style-jako-3495', 'style-jako-3539', 'style-jako-3725', 'style-jako-3807', 'style-koja-3924', 'style-koja-4103', 'style-koja-4264', 'style-koja-4398', 'style-koja-4456', 'style-koja-4524', 'style-koja-4533', 'style-koja-4549', 'style-koja-4610', 'style-koja-4641', 'style-koja-4680', 'style-koja-4766', 'style-koja-4833', 'style-zhko-5282', 'style-zhko-5859', 'style-kozh-6505', 'style-kozh-6525', 'style-kozh-6529', 'style-kozh-6532', 'style-kozh-6561', 'style-kozh-6641', 'style-kozh-6774', 'style-kozh-6903', 'style-kozh-6940', 'style-kozh-6951', 'style-kozh-7141', 'style-kozh-7196', 'style-kozh-7328', 'style-kozh-7630', 'style-kozh-7731']
7800


In [ ]:
merged_lb.to_csv('./gpt_dpo_linebreak_merged.csv', index=False)
# merged_pn.to_csv('./gpt_dpo_propernoun_merged.csv', index=False)
# merged_st.to_csv('./gpt_dpo_style_merged.csv', index=False)

In [ ]:
def check_error_req_and_res(req_df, res_df, error_ids):
    for error_id in error_ids:
        row_req = req_df[req_df['custom_id'] == error_id].iloc[0]
        row_res = res_df[res_df['custom_id'] == error_id].iloc[0]

        request = row_req['body']['messages'][1]['content'].strip()
        response = row_res['response']['body']['choices'][0]['message']['content'].strip()

        print(f"###### Error ID: {error_id}] ######")
        # print(f"[[Request]]\n{request}\n")
        # print("#############################################\n")
        print(f"[[Response]]\n{response}\n")
        print("#############################################\n")

In [ ]:
check_error_req_and_res(req_lb, res_lb, error_ids_lb)

In [ ]:
check_error_req_and_res(req_pn, res_pn, error_ids_pn)

###### Error ID: propernoun-koja-7792] ######
[[Response]]
[PN_MからーNED]
また、질病관리본部では海外旅行の際に、ジカウイルス感染症、デング熱、チクングニア熱などの蚊媒介感染症と細菌性赤痢、腸チフスなど水因性感染症、動物インフルエンザ人体感染症(AI)、中東呼吸器症候群(MERS)などの呼吸器感染症に注意するよう呼びかけた。
[/PN_MAINTAINED]
[PN_TRANSLATED]
また、疾病管理本部では海外旅行の際に、ジカウイルス感染症、デング熱、チクングニア熱などの蚊媒介感染症と細菌性赤痢、腸チフスなど水因性感染症、動物インフルエンザ人体感染症(AI)、中東呼吸器症候群(MERS)などの呼吸器感染症に注意するよう呼びかけた。
[/PN_TRANSLATED]

#############################################



In [ ]:
error_num = 45
check_error_req_and_res(req_st, res_st, [error_ids_st[error_num]])

###### Error ID: style-kozh-7731] ######
[[Response]]
[FORMAL]
难道不是第32秒吗？
[/FORMAL]
[INFORMAL]
不是32秒吗？

#############################################



In [ ]:
df_lb = pd.read_csv('./gpt_dpo_linebreak_merged.csv')
df_pn = pd.read_csv('./gpt_dpo_propernoun_merged.csv')
df_st = pd.read_csv('./gpt_dpo_style_merged.csv')

In [ ]:
df_pn_filtered = df_pn[df_pn['out_src'].notnull() & df_pn['out_tgt'].notnull()]
df_pn_filtered = df_pn_filtered.reset_index(drop=True)
display(df_pn_filtered.head())
print(len(df_pn_filtered))

,id,direction,in_src,in_tgt,out_src,out_tgt
0,propernoun-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...","영부인 회동에는 Angola, Kenya, Nigeria, Tanzania, Zam...","영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
1,propernoun-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 '시티 하베스트 비드 2019'에 카페...
2,propernoun-enko-10,en-ko,"Coach Jung Jung-yong, who led the national tea...",FIFA U-20 월드컵 준우승을 차지한 U-20 대표팀 정정용 감독이 17일 인천...,FIFA U-20 World Cup 준우승을 차지한 U-20 대표팀 Jung Jun...,FIFA U-20 월드컵 준우승을 차지한 U-20 대표팀 정정용 감독이 17일 인천...
3,propernoun-enko-12,en-ko,"In terms of the identity of music, Western mus...","음악의 정체성 문제에 있어서, 양악은 한국의 음악으로서 정체성 논의의 대상이 되지 ...","음악의 정체성 문제에 있어서, Western music는 한국의 음악으로서 정체성 ...","음악의 정체성 문제에 있어서, 서양음악은 한국의 음악으로서 정체성 논의의 대상이 되..."
4,propernoun-enko-26,en-ko,"""Hey Hey Hey"", the love theme of Lee Soo-yeon ...",극중 주인공인 이수연(이제훈)과 한여름(채수빈)의 러브테마인 “이봐 이봐 이봐”는 ...,극중 주인공인 Lee Soo-yeon(Lee Je-hoon)과 Han Yeo-reu...,극중 주인공인 이수연(이제훈)과 한여름(채수빈)의 러브테마인 “이봐 이봐 이봐”는 ...


3442


In [ ]:
def check_df(df, data_num):
    print(f"[Source Original]\n{df['in_src'].iloc[data_num]}")
    print(f"[Target Original]\n{df['in_tgt'].iloc[data_num]}")
    print('---')
    print(f"[Source Maintained]\n{df['out_src'].iloc[data_num]}")
    print(f"[Target Translated]\n{df['out_tgt'].iloc[data_num]}")

In [ ]:
import re

pattern_dict = {
    'en': 'a-zA-Z',
    'ko': '가-힣',
    'zh': '\u4e00-\u9fff',
    'ja': '\u3040-\u309f\u30a0-\u30ff\u4e00-\u9fff'
}
pattern_num = '0-9'

def filter_pn(df):
    original_len = len(df)
    print(f"Original data size: {original_len}")
    for idx, row in df.iterrows():
        src_lang, tgt_lang = row['direction'].split('-')
        pattern_src = rf'^[{pattern_dict[src_lang]}{pattern_num}\s\W]+$'
        pattern_tgt = rf'^[{pattern_dict[tgt_lang]}{pattern_num}\s\W]+$'
        if bool(re.match(pattern_src, row['out_src']) or re.match(pattern_tgt, row['out_src'])):
            df.drop(index=idx, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    print(f"Filtered data size: {len(df)} (Removed {original_len - len(df)} rows)")

    return df

In [ ]:
data_num = 29
check_df(df_pn, data_num)

[Source Original]
Objective theory, subjective theory, compromise theory, Frank's formula, and normative theory have been argued as theories on the criteria for judgment of arbitrariness.
[Target Original]
자의성 판단 기준에 관한 학설로는 객관설, 주관설, 절충설, 프랑크 공식, 규범설 등이 주장되고 있다.
---
[Source Maintained]
자의성 판단 기준에 관한 학설로는 객관설, 주관설, 절충설, Frank 공식, 규범설 등이 주장되고 있다.
[Target Translated]
자의성 판단 기준에 관한 학설로는 객관설, 주관설, 절충설, 프랑크 공식, 규범설 등이 주장되고 있다.


In [ ]:
df_pn_filtered_error = filter_pn(df_pn_filtered)
display(df_pn_filtered_error.head())

Original data size: 3442
Filtered data size: 3350 (Removed 92 rows)


,id,direction,in_src,in_tgt,out_src,out_tgt
0,propernoun-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...","영부인 회동에는 Angola, Kenya, Nigeria, Tanzania, Zam...","영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
1,propernoun-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 '시티 하베스트 비드 2019'에 카페...
2,propernoun-enko-10,en-ko,"Coach Jung Jung-yong, who led the national tea...",FIFA U-20 월드컵 준우승을 차지한 U-20 대표팀 정정용 감독이 17일 인천...,FIFA U-20 World Cup 준우승을 차지한 U-20 대표팀 Jung Jun...,FIFA U-20 월드컵 준우승을 차지한 U-20 대표팀 정정용 감독이 17일 인천...
3,propernoun-enko-12,en-ko,"In terms of the identity of music, Western mus...","음악의 정체성 문제에 있어서, 양악은 한국의 음악으로서 정체성 논의의 대상이 되지 ...","음악의 정체성 문제에 있어서, Western music는 한국의 음악으로서 정체성 ...","음악의 정체성 문제에 있어서, 서양음악은 한국의 음악으로서 정체성 논의의 대상이 되..."
4,propernoun-enko-26,en-ko,"""Hey Hey Hey"", the love theme of Lee Soo-yeon ...",극중 주인공인 이수연(이제훈)과 한여름(채수빈)의 러브테마인 “이봐 이봐 이봐”는 ...,극중 주인공인 Lee Soo-yeon(Lee Je-hoon)과 Han Yeo-reu...,극중 주인공인 이수연(이제훈)과 한여름(채수빈)의 러브테마인 “이봐 이봐 이봐”는 ...


In [ ]:
df_pn_filtered_error['direction'].value_counts()

direction
zh-ko    720
ko-zh    623
ja-ko    598
en-ko    484
ko-en    465
ko-ja    460
Name: count, dtype: int64

In [ ]:
def filter_src_tgt_duplicate(df):
    original_len = len(df)
    for idx, row in df.iterrows():
        if row['out_src'] == row['out_tgt']:
            df.drop(index=idx, inplace=True)
    df.reset_index(drop=True, inplace=True)

    print(f"Filtered data size: {len(df)} (Removed {original_len - len(df)} rows)")

    return df

In [ ]:
df_pn_filtered_error_dup = filter_src_tgt_duplicate(df_pn_filtered_error)
display(df_pn_filtered_error_dup.head())

Filtered data size: 3278 (Removed 72 rows)


,id,direction,in_src,in_tgt,out_src,out_tgt
0,propernoun-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...","영부인 회동에는 Angola, Kenya, Nigeria, Tanzania, Zam...","영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
1,propernoun-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 '시티 하베스트 비드 2019'에 카페...
2,propernoun-enko-10,en-ko,"Coach Jung Jung-yong, who led the national tea...",FIFA U-20 월드컵 준우승을 차지한 U-20 대표팀 정정용 감독이 17일 인천...,FIFA U-20 World Cup 준우승을 차지한 U-20 대표팀 Jung Jun...,FIFA U-20 월드컵 준우승을 차지한 U-20 대표팀 정정용 감독이 17일 인천...
3,propernoun-enko-12,en-ko,"In terms of the identity of music, Western mus...","음악의 정체성 문제에 있어서, 양악은 한국의 음악으로서 정체성 논의의 대상이 되지 ...","음악의 정체성 문제에 있어서, Western music는 한국의 음악으로서 정체성 ...","음악의 정체성 문제에 있어서, 서양음악은 한국의 음악으로서 정체성 논의의 대상이 되..."
4,propernoun-enko-26,en-ko,"""Hey Hey Hey"", the love theme of Lee Soo-yeon ...",극중 주인공인 이수연(이제훈)과 한여름(채수빈)의 러브테마인 “이봐 이봐 이봐”는 ...,극중 주인공인 Lee Soo-yeon(Lee Je-hoon)과 Han Yeo-reu...,극중 주인공인 이수연(이제훈)과 한여름(채수빈)의 러브테마인 “이봐 이봐 이봐”는 ...


In [ ]:
df_pn_filtered_error_dup['direction'].value_counts()

direction
zh-ko    692
ko-zh    605
ja-ko    587
en-ko    475
ko-en    465
ko-ja    454
Name: count, dtype: int64

In [ ]:
df_pn_filtered_error_dup.to_csv('./gpt_dpo_propernoun_merged_filtered.csv', index=False)

In [ ]:
df_lb = pd.read_csv('./gpt_dpo_linebreak_merged.csv')
df_pn = pd.read_csv('./gpt_dpo_propernoun_merged_filtered.csv')
df_st = pd.read_csv('./gpt_dpo_style_merged.csv')

display(df_lb.head(3))
display(df_pn.head(3))
display(df_st.head(3))

,id,direction,in_src,in_tgt,out_src,out_tgt
0,linebreak-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...
1,linebreak-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다."
2,linebreak-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상..."


,id,direction,in_src,in_tgt,out_tgt_maintained,out_tgt_translated
0,propernoun-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...","영부인 회동에는 Angola, Kenya, Nigeria, Tanzania, Zam...","영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
1,propernoun-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,지난해 10월에 있었던 유명 요리 자선행사인 '시티 하베스트 비드 2019'에 카페...
2,propernoun-enko-10,en-ko,"Coach Jung Jung-yong, who led the national tea...",FIFA U-20 월드컵 준우승을 차지한 U-20 대표팀 정정용 감독이 17일 인천...,FIFA U-20 World Cup 준우승을 차지한 U-20 대표팀 Jung Jun...,FIFA U-20 월드컵 준우승을 차지한 U-20 대표팀 정정용 감독이 17일 인천...


,id,direction,in_src,in_tgt,out_tgt_formal,out_tgt_informal
0,style-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,블록체인이란 모든 FFF를 안전하게 기록한 거래 정보를 하나의 데이터베이스 장부로 ...,블록체인이란 모든 FFF를 안전하게 기록한 거래 정보 같은 데이터베이스 장부야.
1,style-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.",그러나 신장 생검이 pSS 환자의 최소 변화 질환에 대한 진단을 확인하였습니다.,근데 신장 생검으로 pSS 환자의 최소 변화 질환 진단이 확실해졌어요.
2,style-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...","회의에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 영부인들...","이번 회의에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아의 영부인들과 영국 총리..."


In [ ]:
print(df_lb['in_src'][4])
print(df_lb['in_tgt'][4])
print(df_lb['out_src'][4])
print(df_lb['out_tgt'][4])
print(df_pn['out_tgt_maintained'][1])
print(df_pn['out_tgt_translated'][1])
print(df_st['out_tgt_formal'][4])
print(df_st['out_tgt_informal'][4])

On behalf of Cafe Boulud NYC at CITY HARVEST BID 2019, a famous culinary charity event held in October last year, he presented Charcuterie, a traditional French dish.
지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 2019'에 Cafe Boulud NYC를 대표해 나가 프랑스 대표 전통요리인 샤퀴테리(Charcuterie) 요리를 선보였다.
On behalf of Cafe Boulud NYC at CITY HARVEST BID 2019, a famous culinary 
charity event held in October last year, he presented Charcuterie, a traditional French dish.
지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 2019'에 
Cafe Boulud NYC를 대표해 나가 프랑스 대표 전통요리인 샤퀴테리(Charcuterie) 요리를 선보였다.
지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 2019'에 Cafe Boulud NYC를 대표해 나가 프랑스 대표 전통요리인 Charcuterie 요리를 선보였다.
지난해 10월에 있었던 유명 요리 자선행사인 '시티 하베스트 비드 2019'에 카페 불루드 NYC를 대표해 나가 프랑스 대표 전통요리인 샤퀴테리 요리를 선보였다.
그는 지난해 10월에 열린 유명한 요리 자선 행사인 'CITY HARVEST BID 2019'에서 Cafe Boulud NYC를 대표하여 프랑스의 전통 요리인 샤퀴테리(Charcuterie)를 선보였다.
그는 작년 10월에 열린 유명한 요리 자선 행사 'CITY HARVEST BID 2019'에서 Cafe Boulud NYC를 대신해 프랑스 전통 음식인 샤퀴테리(Charcuterie)를 보여줬어요.


In [ ]:
df_lb = pd.read_csv('./gpt_dpo_linebreak_modification_merged.csv')

In [ ]:
df_lb

,id,direction,in_src,in_tgt,out_src,out_tgt
0,linebreak-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...
1,linebreak-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다."
2,linebreak-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
3,linebreak-enko-4,en-ko,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...
4,linebreak-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...
...,...,...,...,...,...,...
3894,linebreak-kozh-3895,ko-zh,하지만 여당으로선 해당 안도 현실성이 없다고 보고 '250:50안'으로 가닥을 잡았...,"但是,执政党认为该方案不具备现实性,决定采用""250:50方案""来理顺纠结事项。",단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。
3895,linebreak-kozh-3896,ko-zh,단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。,수고하세요\n그때 뵐게요.,辛苦了，\n到时候再见吧。
3896,linebreak-kozh-3897,ko-zh,수고하세요 그때 뵐게요.,"辛苦了,到时候再见吧。",와인 저장 장비와 와인 파이프라인을 \n정기적으로 청소하십시오.,定期对储酒设备、\n输酒管道进行清洗。
3897,linebreak-kozh-3898,ko-zh,와인 저장 장비와 와인 파이프라인을 정기적으로 청소하십시오.,定期对储酒设备、输酒管道进行清洗。,그 때문에 자신의 소신을 세우고 재물을 \n크게 벌어들일 가능성이 아주 커.,因此，树立自己的信念，赚大钱的\n可能性非常大。


In [ ]:
from collections import defaultdict

df_lb = df_lb_merged_filtered.copy()

df_lb_reflected = defaultdict(list)
df_lb_ignored = defaultdict(list)
for idx, row in df_lb.iterrows():
    src_lang, tgt_lang = row['direction'].split('-')
    trans_lang = src_lang if tgt_lang == 'ko' else tgt_lang

    df_lb_reflected['id'].append(row['id'])
    df_lb_reflected['src'].append(row['out_src'])
    df_lb_reflected['tgt'].append(row['out_tgt'])
    df_lb_reflected['data_source'].append(f'base-1m-{trans_lang}')
    df_lb_reflected['direction'].append(row['direction'])
    df_lb_reflected['dpo_info'].append('lb-reflected')

    df_lb_ignored['id'].append(row['id'])
    df_lb_ignored['src'].append(row['out_src'])
    df_lb_ignored['tgt'].append(row['in_tgt'])
    df_lb_ignored['data_source'].append(f'base-1m-{trans_lang}')
    df_lb_ignored['direction'].append(row['direction'])
    df_lb_ignored['dpo_info'].append('lb-ignored')

df_lb_reflected = pd.DataFrame(df_lb_reflected)
df_lb_ignored = pd.DataFrame(df_lb_ignored)

df_lb_dpo = pd.concat([df_lb_reflected, df_lb_ignored], ignore_index=True)

In [ ]:
display(df_lb_dpo)
print(len(df_lb_dpo))

,id,src,tgt,data_source,direction,dpo_info
0,linebreak-enko-1,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,linebreak-enko-2,"However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",base-1m-en,en-ko,lb-reflected
3,linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...,base-1m-en,en-ko,lb-reflected
4,linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
6477,linebreak-kozh-3893,"주요 경제 법안인 탄력근로제 확대와 빅데이터 3법,\n유치원 3법 등도 처리 여부가...",主要经济法案弹性劳动制规模规模扩大和大数据3法、幼儿园3法等的处理也是很灵活的。,base-1m-zh,ko-zh,lb-ignored
6478,linebreak-kozh-3894,"하지만 여당으로선 해당 안도 현실성이 없다고 보고,\n'250:50안'으로 가닥을 ...","但是,执政党认为该方案不具备现实性,决定采用""250:50方案""来理顺纠结事项。",base-1m-zh,ko-zh,lb-ignored
6479,linebreak-kozh-3895,단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。,base-1m-zh,ko-zh,lb-ignored
6480,linebreak-kozh-3896,수고하세요\n그때 뵐게요.,"辛苦了,到时候再见吧。",base-1m-zh,ko-zh,lb-ignored


6482


In [ ]:
bad_data_idx = df_lb_dpo[df_lb_dpo['tgt'] == '>어어!'].index[0]

# display(df_lb_dpo.loc[bad_data_idx:len(df_lb_dpo)-2, 'tgt'])
# print(len(df_lb_dpo.loc[bad_data_idx:len(df_lb_dpo)-2, 'tgt']))
# display(df_lb_dpo.loc[bad_data_idx+1:, 'tgt'])
# print(len(df_lb_dpo.loc[bad_data_idx+1:, 'tgt']))
df_lb_dpo.loc[bad_data_idx:len(df_lb_dpo)-2, 'tgt'] = df_lb_dpo.loc[bad_data_idx+1:, 'tgt'].tolist()
df_lb_dpo = df_lb_dpo.iloc[:-1]

df_lb_dpo.head()

,id,src,tgt,data_source,direction,dpo_info
0,linebreak-enko-1,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,linebreak-enko-2,"However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",base-1m-en,en-ko,lb-reflected
3,linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...,base-1m-en,en-ko,lb-reflected
4,linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected


In [ ]:
df_lb_dpo.to_csv('./gpt_dpo_linebreak_modification.csv', index=False)

In [ ]:
df_pn_maintained = defaultdict(list)
df_pn_translated = defaultdict(list)
for idx, row in df_pn.iterrows():
    src_lang, tgt_lang = row['direction'].split('-')
    trans_lang = src_lang if tgt_lang == 'ko' else tgt_lang

    df_pn_maintained['id'].append(row['id'])
    df_pn_maintained['src'].append(row['in_src'])
    df_pn_maintained['tgt'].append(row['out_tgt_maintained'])
    df_pn_maintained['data_source'].append(f'base-1m-{trans_lang}')
    df_pn_maintained['direction'].append(row['direction'])
    df_pn_maintained['dpo_info'].append('pn-maintained')

    df_pn_translated['id'].append(row['id'])
    df_pn_translated['src'].append(row['in_src'])
    df_pn_translated['tgt'].append(row['out_tgt_translated'])
    df_pn_translated['data_source'].append(f'base-1m-{trans_lang}')
    df_pn_translated['direction'].append(row['direction'])
    df_pn_translated['dpo_info'].append('pn-translated')

df_pn_maintained = pd.DataFrame(df_pn_maintained)
df_pn_translated = pd.DataFrame(df_pn_translated)

df_pn_dpo = pd.concat([df_pn_maintained, df_pn_translated], ignore_index=True)

In [ ]:
display(df_pn_dpo)
print(len(df_pn_dpo))

,id,src,tgt,data_source,direction,dpo_info
0,propernoun-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 Angola, Kenya, Nigeria, Tanzania, Zam...",base-1m-en,en-ko,pn-maintained
1,propernoun-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,pn-maintained
2,propernoun-enko-10,"Coach Jung Jung-yong, who led the national tea...",FIFA U-20 World Cup 준우승을 차지한 U-20 대표팀 Jung Jun...,base-1m-en,en-ko,pn-maintained
3,propernoun-enko-12,"In terms of the identity of music, Western mus...","음악의 정체성 문제에 있어서, Western music는 한국의 음악으로서 정체성 ...",base-1m-en,en-ko,pn-maintained
4,propernoun-enko-26,"""Hey Hey Hey"", the love theme of Lee Soo-yeon ...",극중 주인공인 Lee Soo-yeon(Lee Je-hoon)과 Han Yeo-reu...,base-1m-en,en-ko,pn-maintained
...,...,...,...,...,...,...
6551,propernoun-kozh-13175,여기에 경기도에서 총선 불출마 폭이 얼마나 될지도 관심거리다.,"此外,京畿道不参加议会选举的幅度有多大也备受关注。",base-1m-zh,ko-zh,pn-translated
6552,propernoun-kozh-13186,포르쉐 디자인을 총괄하는 수석디자이너인 핑키 라이는 포르쉐의 자동차디자인 철학을 소...,负责保时捷设计的首席设计师Pinky Lai介绍了保时捷的汽车设计哲学。,base-1m-zh,ko-zh,pn-translated
6553,propernoun-kozh-13187,마치 '더 로드' 속 아버지와 아들이 남쪽을 향해 끝없이 걸어갔던 것처럼 말이다.,仿佛《道路》中的父亲和儿子向着南方不停地走一样。,base-1m-zh,ko-zh,pn-translated
6554,propernoun-kozh-13189,해외 거대 플랫폼 업체들이 코로나19 국면에서 국내 이용자들을 홀대해 빈축을 사고 있다.,"海外大型平台企业在新冠疫情局面下忽视了韩国用户,引起了人们的嫌弃。",base-1m-zh,ko-zh,pn-translated


6556


In [ ]:
df_st_formal = defaultdict(list)
df_st_informal = defaultdict(list)
for idx, row in df_st.iterrows():
    src_lang, tgt_lang = row['direction'].split('-')
    trans_lang = src_lang if tgt_lang == 'ko' else tgt_lang

    df_st_formal['id'].append(row['id'])
    df_st_formal['src'].append(row['in_src'])
    df_st_formal['tgt'].append(row['out_tgt_formal'])
    df_st_formal['data_source'].append(f'base-1m-{trans_lang}')
    df_st_formal['direction'].append(row['direction'])
    df_st_formal['dpo_info'].append('st-formal')

    df_st_informal['id'].append(row['id'])
    df_st_informal['src'].append(row['in_src'])
    df_st_informal['tgt'].append(row['out_tgt_informal'])
    df_st_informal['data_source'].append(f'base-1m-{trans_lang}')
    df_st_informal['direction'].append(row['direction'])
    df_st_informal['dpo_info'].append('st-informal')

df_st_formal = pd.DataFrame(df_st_formal)
df_st_informal = pd.DataFrame(df_st_informal)

df_st_dpo = pd.concat([df_st_formal, df_st_informal], ignore_index=True)


In [ ]:
st_inf = df_st_dpo[df_st_dpo['dpo_info'] == 'st-informal']
st_inf[st_inf['direction'] == 'ko-en']

,id,src,tgt,data_source,direction,dpo_info
9100,style-koen-1301,"비타민D가 인체 세포의 DNA와 결합해 다발성 경화증과 당뇨, 암 같은 질병과 관련...","영국의 BBB1과 BBB2 인터넷 기사에 따르면, 비타민 D가 사람의 세포 DNA와...",base-1m-en,ko-en,st-informal
9101,style-koen-1302,근세에 독일을 비롯한 대륙 유럽 법계에서는 국가가 공익의 실현을 독점하면서 민간 공...,"요즘 독일 같은 대륙 유럽에서는 국가는 공익을 독차지하고, 민간 공익 단체는 국가 ...",base-1m-en,ko-en,st-informal
9102,style-koen-1303,또한 본 카탈로그에는 AAA님과의 작업 당시 진행한 인터뷰도 포함되어 있습니다.,이 카탈로그에는 AAA랑 일할 때 한 인터뷰도 들어 있어요.,base-1m-en,ko-en,st-informal
9103,style-koen-1304,그리고 오늘 저희는 BBB 회사로부터 LED TV에 대한 오퍼가 있다는 이메일을 받...,오늘 BBB 회사에서 LED TV 제안 왔다고 이메일이 왔어요.,base-1m-en,ko-en,st-informal
9104,style-koen-1305,원이 세 개의 주어진 원에 접하는 조건은 원들이 외부 또는 내부적으로 접하기 때문에...,세 개의 원에 접하는 원이 어떤 조건을 만족해야 하는지는 두 원의 중심 간 거리가 ...,base-1m-en,ko-en,st-informal
...,...,...,...,...,...,...
10395,style-koen-2596,이하에서는 X1 → X6 → X8 로 데이터 패킷이 전달되는 것으로 가정한다.,"아래에서는 데이터 패킷이 X1에서 X6, 그리고 X8로 차례로 간다고 보고 있어요.",base-1m-en,ko-en,st-informal
10396,style-koen-2597,"이후 더욱 진전된 논의가 이루어 졌으며, 우리나라가 왜 CSR을 부담해야 하는지에 ...","그 이후에 더 깊은 얘기가 오갔고, 우리나라가 왜 CSR을 부담해야 하는지 이유도 ...",base-1m-en,ko-en,st-informal
10397,style-koen-2598,"네, 뭘 논의하고 싶으신가요?","네, 뭐 얘기하고 싶으세요?",base-1m-en,ko-en,st-informal
10398,style-koen-2599,통화를 끊고 A씨는 홧김에 자고 있던 범행을 저질렀지만 다행히 B씨는 생명에 지장이...,"전화 끊고 A씨가 화나서 잠들면서 사고쳤는데, 다행히 B씨는 괜찮다고 하더라구요.",base-1m-en,ko-en,st-informal


In [ ]:
display(df_st_dpo)
print(len(df_st_dpo))

,id,src,tgt,data_source,direction,dpo_info
0,style-enko-1,Blockchain is the transaction information that...,블록체인이란 모든 FFF를 안전하게 기록한 거래 정보를 하나의 데이터베이스 장부로 ...,base-1m-en,en-ko,st-formal
1,style-enko-2,"However, renal biopsy confirms diagnosis for m...",그러나 신장 생검이 pSS 환자의 최소 변화 질환에 대한 진단을 확인하였습니다.,base-1m-en,en-ko,st-formal
2,style-enko-3,The meeting will also be attended by first lad...,"회의에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 영부인들...",base-1m-en,en-ko,st-formal
3,style-enko-4,The collection of neural signals by the cuff-t...,커프형 전극을 통한 신경 신호 수집은 FNS 시스템에서 필요한 정보 제공 및 피드백...,base-1m-en,en-ko,st-formal
4,style-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,그는 지난해 10월에 열린 유명한 요리 자선 행사인 'CITY HARVEST BID...,base-1m-en,en-ko,st-formal
...,...,...,...,...,...,...
15595,style-kozh-7796,"저는 이제 밤에도 핸드폰을 끄지 않아요, 일이 있어 저를 찾을 것 같아 걱정됩니다.",我现在晚上手机都不关机，怕有人找我有事儿。,base-1m-zh,ko-zh,st-informal
15596,style-kozh-7797,현재까지 안드로이드 기반 스마트폰에만 적용된 이 기능은 1.5배 빠르게 보기부터 0...,现在这个功能只能在安卓手机上用，能调节速度从1.5倍快放到0.5倍慢放哦。,base-1m-zh,ko-zh,st-informal
15597,style-kozh-7798,이것의 유입을 방지하고 중국 과일의 생산 안전을 보호하기 위해 중국은 미국 과일 수...,为了不让美国水果进来，保护咱们的水果生产，中国决定不让美国水果进来。,base-1m-zh,ko-zh,st-informal
15598,style-kozh-7799,해파리 팀원을 빼앗겼다고?,你是说水母队的成员被抢走了？,base-1m-zh,ko-zh,st-informal


15600


In [ ]:
# lb, st 전처리 필요(언어 오류, 중복 등)

In [ ]:
df_dpo = pd.concat([df_lb_dpo, df_pn_dpo, df_st_dpo], ignore_index=True)
df_dpo['id'] = df_dpo['id'].apply(lambda x: f'dpo-{x}')
display(df_dpo)
print(len(df_dpo))

,id,src,tgt,data_source,direction,dpo_info
0,dpo-linebreak-enko-1,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상...",base-1m-en,en-ko,lb-reflected
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...,base-1m-en,en-ko,lb-reflected
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
29951,dpo-style-kozh-7796,"저는 이제 밤에도 핸드폰을 끄지 않아요, 일이 있어 저를 찾을 것 같아 걱정됩니다.",我现在晚上手机都不关机，怕有人找我有事儿。,base-1m-zh,ko-zh,st-informal
29952,dpo-style-kozh-7797,현재까지 안드로이드 기반 스마트폰에만 적용된 이 기능은 1.5배 빠르게 보기부터 0...,现在这个功能只能在安卓手机上用，能调节速度从1.5倍快放到0.5倍慢放哦。,base-1m-zh,ko-zh,st-informal
29953,dpo-style-kozh-7798,이것의 유입을 방지하고 중국 과일의 생산 안전을 보호하기 위해 중국은 미국 과일 수...,为了不让美国水果进来，保护咱们的水果生产，中国决定不让美国水果进来。,base-1m-zh,ko-zh,st-informal
29954,dpo-style-kozh-7799,해파리 팀원을 빼앗겼다고?,你是说水母队的成员被抢走了？,base-1m-zh,ko-zh,st-informal


29956


In [ ]:
print(len(df_dpo[df_dpo['dpo_info'].str.startswith('lb')]))
print(len(df_dpo[df_dpo['dpo_info'].str.startswith('pn')]))
print(len(df_dpo[df_dpo['dpo_info'].str.startswith('st')]))

7800
6556
15600


In [ ]:
df_dpo.to_csv('./dpo_unpreprocessed.csv', index=False)

In [ ]:
df_dpo = pd.read_csv('./dpo_unpreprocessed.csv')

In [ ]:
print(len(df_dpo))
df_dpo = df_dpo[(~df_dpo['dpo_info'].str.startswith('st')) | (~df_dpo['direction'].str.endswith('en'))]
print(len(df_dpo))

29956
27356


In [ ]:
df_st_koen_req = pd.read_json('./gpt_dpo_style_request_koen.jsonl', lines=True)
display(df_st_koen_req.head())
print(len(df_st_koen_req))

,custom_id,method,url,body
0,style-koen-1,POST,/v1/chat/completions,"{'model': 'gpt-4o-2024-08-06', 'messages': [{'..."
1,style-koen-2,POST,/v1/chat/completions,"{'model': 'gpt-4o-2024-08-06', 'messages': [{'..."
2,style-koen-3,POST,/v1/chat/completions,"{'model': 'gpt-4o-2024-08-06', 'messages': [{'..."
3,style-koen-4,POST,/v1/chat/completions,"{'model': 'gpt-4o-2024-08-06', 'messages': [{'..."
4,style-koen-5,POST,/v1/chat/completions,"{'model': 'gpt-4o-2024-08-06', 'messages': [{'..."


1300


In [ ]:
df_st_koen_res = pd.read_json('./gpt_dpo_style_response_koen.jsonl', lines=True)
display(df_st_koen_res.head())
print(len(df_st_koen_res))

,id,custom_id,response,error
0,batch_req_6BsnQ9uPmKqwAh9cInEwH2H0,style-koen-1,"{'status_code': 200, 'request_id': '3a740158fa...",NaN
1,batch_req_Gg5p3g4VYQXRGOVOazjrVYL6,style-koen-2,"{'status_code': 200, 'request_id': '989f69ba92...",NaN
2,batch_req_aQ16E0GORB4YH1Z8vThp2p7V,style-koen-3,"{'status_code': 200, 'request_id': '10decc4ea8...",NaN
3,batch_req_WW2jQfg9HnLXp0JaSgXic60V,style-koen-4,"{'status_code': 200, 'request_id': 'c32267bd44...",NaN
4,batch_req_GtC7ebZbVH7eTxMaXpmRpMng,style-koen-5,"{'status_code': 200, 'request_id': '8c1624de7d...",NaN


1300


In [ ]:
df_st_koen, error_ids_st = merge_req_and_res_for_style(df_st_koen_req, df_st_koen_res)
display(df_st_koen.head())
print(len(df_st_koen))

Merging Style DF: 100%|██████████| 1300/1300 [00:00<00:00, 6992.36it/s]


,id,direction,in_src,in_tgt,out_src,out_tgt
0,style-koen-1,ko-en,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information that...,Blockchain is defined as a ledger database tha...,"So basically, blockchain's a database that kee..."
1,style-koen-2,ko-en,"그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","근데, 신장 생검은 pSS 환자의 최소 변화 질환 진단을 확실히 해줘요."
2,style-koen-3,ko-en,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,The gathering will also be attended by the Fir...,Joining the meeting will be the wives of vario...
3,style-koen-4,ko-en,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,Collecting neural signals through cuff-type el...,Using cuff-type electrodes to gather nerve sig...
4,style-koen-5,ko-en,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,"Last October, at the renowned culinary charity...","Last October, at the famous food charity event..."


1300


In [ ]:
df_st_koen.rename(columns={'out_src': 'out_tgt_formal', 'out_tgt': 'out_tgt_informal'}, inplace=True)
display(df_st_koen.head())

,id,direction,in_src,in_tgt,out_tgt_formal,out_tgt_informal
0,style-koen-1,ko-en,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information that...,Blockchain is defined as a ledger database tha...,"So basically, blockchain's a database that kee..."
1,style-koen-2,ko-en,"그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","근데, 신장 생검은 pSS 환자의 최소 변화 질환 진단을 확실히 해줘요."
2,style-koen-3,ko-en,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,The gathering will also be attended by the Fir...,Joining the meeting will be the wives of vario...
3,style-koen-4,ko-en,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,Collecting neural signals through cuff-type el...,Using cuff-type electrodes to gather nerve sig...
4,style-koen-5,ko-en,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,"Last October, at the renowned culinary charity...","Last October, at the famous food charity event..."


In [ ]:
import re

pattern_dict = {
    'en': 'a-zA-Z',
    'ko': '가-힣',
    'zh': '\u4e00-\u9fff',
    'ja': '\u3040-\u309f\u30a0-\u30ff\u4e00-\u9fff'
}
pattern_num = '0-9'

def filter_st(df):
    filtered_df = df.copy()
    print(f"Original data size: {len(df)}")
    for idx, row in df.iterrows():
        src_lang, tgt_lang = row['direction'].split('-')
        pattern_src = rf'^[{pattern_dict[src_lang]}{pattern_num}\s\W]+$'
        if bool(re.match(pattern_src, row['out_tgt_formal']) or re.match(pattern_src, row['out_tgt_informal'])):
            print(f"Removed data: {row['out_tgt_formal']}")
            filtered_df.drop(index=idx, inplace=True)
    filtered_df.reset_index(drop=True, inplace=True)
    
    print(f"Filtered data size: {len(filtered_df)} (Removed {len(df) - len(filtered_df)} rows)")

    return filtered_df

In [ ]:
df_st_koen_filtered = filter_st(df_st_koen)
display(df_st_koen_filtered.head())

Original data size: 1300
Removed data: 이러한 구조를 제공하기 위한 쇼트 아크형 방전 램프는 발광관, 발광관의 양단에 연결된 밀봉관, 발광관 내에 대향으로 배열된 한 쌍의 전극, 그리고 전극을 지지하는 전극심선을 포함하는데, 밀봉관 내에 전극심선 지지체를 배치해도 밀봉관을 밀봉할 때 융합된 유리 재료가 전극심선 지지체 전면에 처져 융착되지 않아, 그 부분에서 균열 발생을 방지한다. 전극심선 지지체의 외측 둘레에 감겨진 완충박은 전극심선 지지체의 전단면보다 발광관 측으로 더 돌출되어 있다.
Removed data: >정말로 달콤한가요?
Removed data: 소형견 및 중형견의 사용을 권장합니다.
Removed data: 그로 인해 촬영 현장에 있던 다른 사람들 또한 솔비가 예술을 하는 것이 큰 실수라는 이야기를 이끌기 시작했습니다.
Removed data: 또한, 그들은 서로에게 인사를 잘하고 동방의 예의 있는 나라로 불립니다.
Removed data: 당신의 도움 없이는 이 일을 해낼 수 없었습니다.
Removed data: 이어지는 절차로, 전자 기기는 휴리스틱 작업 제안에 해당하는 제안 어포던스를 표시할 수 있습니다.
Removed data: 그는 안전 장비 판매에 전념해 온 전문가이기도 합니다.
Removed data: 저는 신용 대출을 받지 않고 기다렸다면 이익을 얻을 수 있었을 것이라는 생각에 후회가 밀려왔습니다.
Removed data: '팀 벤투'의 젊은 선수 황희찬(함부르크)은 남은 경기들에서 자신의 강점을 살린 속도감 있는 공격으로 상대 수비라인을 흔들겠다는 결심을 했다.
Removed data: 오늘 이 이메일이 귀하께 도착하기를 바랍니다.
Removed data: 귀하의 비즈니스 수익을 위한 저축 계좌 개설 관련 문의가 접수되었습니다.
Removed data: 개폐 시 및 개방 시 비수납 공간 측으로의 접이문 돌출 크기를 줄이면서 수납 공간을 효율적으로 활용할 수 있는 수납 개구 개폐 구조를 제공합니다. 수

,id,direction,in_src,in_tgt,out_tgt_formal,out_tgt_informal
0,style-koen-1,ko-en,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information that...,Blockchain is defined as a ledger database tha...,"So basically, blockchain's a database that kee..."
1,style-koen-2,ko-en,"그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","근데, 신장 생검은 pSS 환자의 최소 변화 질환 진단을 확실히 해줘요."
2,style-koen-3,ko-en,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,The gathering will also be attended by the Fir...,Joining the meeting will be the wives of vario...
3,style-koen-4,ko-en,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,Collecting neural signals through cuff-type el...,Using cuff-type electrodes to gather nerve sig...
4,style-koen-5,ko-en,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,"Last October, at the renowned culinary charity...","Last October, at the famous food charity event..."


In [ ]:
df_st_koen_filtered

,id,direction,in_src,in_tgt,out_tgt_formal,out_tgt_informal
0,style-koen-1,ko-en,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information that...,Blockchain is defined as a ledger database tha...,"So basically, blockchain's a database that kee..."
1,style-koen-2,ko-en,"그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","근데, 신장 생검은 pSS 환자의 최소 변화 질환 진단을 확실히 해줘요."
2,style-koen-3,ko-en,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,The gathering will also be attended by the Fir...,Joining the meeting will be the wives of vario...
3,style-koen-4,ko-en,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,Collecting neural signals through cuff-type el...,Using cuff-type electrodes to gather nerve sig...
4,style-koen-5,ko-en,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,"Last October, at the renowned culinary charity...","Last October, at the famous food charity event..."
...,...,...,...,...,...,...
1176,style-koen-1296,ko-en,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,"The 'Autumn Marathon Lover's Competition', whi...","The 'Autumn Marathon Enthusiast Competition', ...",The 'Autumn Marathon Lover's Competition' that...
1177,style-koen-1297,ko-en,손에 몇 방울을 떨어뜨리세요.,Apply a few drops on your hand.,Please apply a few drops onto your hand.,Just put a few drops on your hand.
1178,style-koen-1298,ko-en,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,The touch detection electrode TDL is configure...,The touch detection electrode (TDL) is compose...,"The touch detection electrode, or TDL, is made..."
1179,style-koen-1299,ko-en,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","In some countries like Denmark, Malta, Switzer...","In several countries, including Denmark, Malta...","Places like Denmark, Malta, Switzerland, and P..."


In [ ]:
df_dpo = pd.concat([df_dpo, df_st_koen_filtered], ignore_index=True)

In [ ]:
df_dpo

,id,src,tgt,data_source,direction,dpo_info,in_src,in_tgt,out_tgt_formal,out_tgt_informal
0,dpo-linebreak-enko-1,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상...",base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...,base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
28532,style-koen-1296,NaN,NaN,NaN,ko-en,NaN,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,"The 'Autumn Marathon Lover's Competition', whi...","The 'Autumn Marathon Enthusiast Competition', ...",The 'Autumn Marathon Lover's Competition' that...
28533,style-koen-1297,NaN,NaN,NaN,ko-en,NaN,손에 몇 방울을 떨어뜨리세요.,Apply a few drops on your hand.,Please apply a few drops onto your hand.,Just put a few drops on your hand.
28534,style-koen-1298,NaN,NaN,NaN,ko-en,NaN,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,The touch detection electrode TDL is configure...,The touch detection electrode (TDL) is compose...,"The touch detection electrode, or TDL, is made..."
28535,style-koen-1299,NaN,NaN,NaN,ko-en,NaN,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","In some countries like Denmark, Malta, Switzer...","In several countries, including Denmark, Malta...","Places like Denmark, Malta, Switzerland, and P..."


In [ ]:
df_dpo = df_dpo.iloc[:27356]
df_dpo

,id,src,tgt,data_source,direction,dpo_info,in_src,in_tgt,out_tgt_formal,out_tgt_informal
0,dpo-linebreak-enko-1,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상...",base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...,base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
27351,dpo-style-kozh-7796,"저는 이제 밤에도 핸드폰을 끄지 않아요, 일이 있어 저를 찾을 것 같아 걱정됩니다.",我现在晚上手机都不关机，怕有人找我有事儿。,base-1m-zh,ko-zh,st-informal,NaN,NaN,NaN,NaN
27352,dpo-style-kozh-7797,현재까지 안드로이드 기반 스마트폰에만 적용된 이 기능은 1.5배 빠르게 보기부터 0...,现在这个功能只能在安卓手机上用，能调节速度从1.5倍快放到0.5倍慢放哦。,base-1m-zh,ko-zh,st-informal,NaN,NaN,NaN,NaN
27353,dpo-style-kozh-7798,이것의 유입을 방지하고 중국 과일의 생산 안전을 보호하기 위해 중국은 미국 과일 수...,为了不让美国水果进来，保护咱们的水果生产，中国决定不让美国水果进来。,base-1m-zh,ko-zh,st-informal,NaN,NaN,NaN,NaN
27354,dpo-style-kozh-7799,해파리 팀원을 빼앗겼다고?,你是说水母队的成员被抢走了？,base-1m-zh,ko-zh,st-informal,NaN,NaN,NaN,NaN


In [ ]:
df_dpo.drop(columns=['in_src', 'in_tgt', 'out_tgt_formal', 'out_tgt_informal'], inplace=True)
df_dpo

/tmp/ipykernel_1052506/79708636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dpo.drop(columns=['in_src', 'in_tgt', 'out_tgt_formal', 'out_tgt_informal'], inplace=True)


,id,src,tgt,data_source,direction,dpo_info
0,dpo-linebreak-enko-1,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상...",base-1m-en,en-ko,lb-reflected
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...,base-1m-en,en-ko,lb-reflected
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
27351,dpo-style-kozh-7796,"저는 이제 밤에도 핸드폰을 끄지 않아요, 일이 있어 저를 찾을 것 같아 걱정됩니다.",我现在晚上手机都不关机，怕有人找我有事儿。,base-1m-zh,ko-zh,st-informal
27352,dpo-style-kozh-7797,현재까지 안드로이드 기반 스마트폰에만 적용된 이 기능은 1.5배 빠르게 보기부터 0...,现在这个功能只能在安卓手机上用，能调节速度从1.5倍快放到0.5倍慢放哦。,base-1m-zh,ko-zh,st-informal
27353,dpo-style-kozh-7798,이것의 유입을 방지하고 중국 과일의 생산 안전을 보호하기 위해 중국은 미국 과일 수...,为了不让美国水果进来，保护咱们的水果生产，中国决定不让美国水果进来。,base-1m-zh,ko-zh,st-informal
27354,dpo-style-kozh-7799,해파리 팀원을 빼앗겼다고?,你是说水母队的成员被抢走了？,base-1m-zh,ko-zh,st-informal


In [ ]:
df_st_formal_koen = defaultdict(list)
df_st_informal_koen = defaultdict(list)
for idx, row in df_st_koen_filtered.iterrows():
    src_lang, tgt_lang = row['direction'].split('-')
    trans_lang = src_lang if tgt_lang == 'ko' else tgt_lang

    df_st_formal_koen['id'].append(row['id'])
    df_st_formal_koen['src'].append(row['in_src'])
    df_st_formal_koen['tgt'].append(row['out_tgt_formal'])
    df_st_formal_koen['data_source'].append(f'base-1m-{trans_lang}')
    df_st_formal_koen['direction'].append(row['direction'])
    df_st_formal_koen['dpo_info'].append('st-formal')

    df_st_informal_koen['id'].append(row['id'])
    df_st_informal_koen['src'].append(row['in_src'])
    df_st_informal_koen['tgt'].append(row['out_tgt_informal'])
    df_st_informal_koen['data_source'].append(f'base-1m-{trans_lang}')
    df_st_informal_koen['direction'].append(row['direction'])
    df_st_informal_koen['dpo_info'].append('st-informal')

df_st_formal_koen = pd.DataFrame(df_st_formal_koen)
df_st_informal_koen = pd.DataFrame(df_st_informal_koen)

df_st_dpo_koen = pd.concat([df_st_formal_koen, df_st_informal_koen], ignore_index=True)


In [ ]:
df_st_dpo_koen

,id,src,tgt,data_source,direction,dpo_info
0,style-koen-1,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is defined as a ledger database tha...,base-1m-en,ko-en,st-formal
1,style-koen-2,"그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,ko-en,st-formal
2,style-koen-3,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The gathering will also be attended by the Fir...,base-1m-en,ko-en,st-formal
3,style-koen-4,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,Collecting neural signals through cuff-type el...,base-1m-en,ko-en,st-formal
4,style-koen-5,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,"Last October, at the renowned culinary charity...",base-1m-en,ko-en,st-formal
...,...,...,...,...,...,...
2357,style-koen-1296,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en,st-informal
2358,style-koen-1297,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en,st-informal
2359,style-koen-1298,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en,st-informal
2360,style-koen-1299,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en,st-informal


In [ ]:
df_dpo = pd.concat([df_dpo, df_st_dpo_koen], ignore_index=True)

In [ ]:
df_dpo

,id,src,tgt,data_source,direction,dpo_info
0,dpo-linebreak-enko-1,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상...",base-1m-en,en-ko,lb-reflected
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...,base-1m-en,en-ko,lb-reflected
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
29713,style-koen-1296,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en,st-informal
29714,style-koen-1297,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en,st-informal
29715,style-koen-1298,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en,st-informal
29716,style-koen-1299,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en,st-informal


In [ ]:
for idx, row in df_dpo.iterrows():
    if not row['id'].startswith('dpo'):
        df_dpo.at[idx, 'id'] = f'dpo-{row["id"]}'

df_dpo

,id,src,tgt,data_source,direction,dpo_info
0,dpo-linebreak-enko-1,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상...",base-1m-en,en-ko,lb-reflected
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...,base-1m-en,en-ko,lb-reflected
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
29713,dpo-style-koen-1296,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en,st-informal
29714,dpo-style-koen-1297,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en,st-informal
29715,dpo-style-koen-1298,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en,st-informal
29716,dpo-style-koen-1299,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en,st-informal


In [ ]:
df_dpo.to_csv('./dpo_unpreprocessed.csv', index=False)

In [ ]:
df_dpo = pd.read_csv('./dpo_unpreprocessed.csv')


In [ ]:
import sys

import pandas as pd
import numpy as np
from openai import OpenAI

sys.path.append('../../inference/codes')
from api_secret import OPENAI_CLIENT_KEY_TMAXNLP

In [ ]:
LANG_TABLE = {
    'en': 'English',
    'ko': '한국어',
    'ja': '日本語',
    'zh': '中文',
}

In [ ]:
class GptGenerator:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)

    def get_gpt_system_prompt(self, src_lang_code='en', tgt_lang_code='ko'):
        src_lang_full = LANG_TABLE[src_lang_code]
        tgt_lang_full = LANG_TABLE[tgt_lang_code]
        gpt_system_instruction = f"""
            You are an assistant for making {src_lang_full}-{tgt_lang_full} text-pair data, for the another translation model.
            You have to modify the given text-pair, by following the task guidelines below.
        """
        gpt_system_instruction = re.sub(r'\s{2,}', '\n', gpt_system_instruction).strip()
        gpt_system_prompt = f"<instruction>\n{gpt_system_instruction}\n</instruction>"
        return gpt_system_prompt

    def generate(self, prompt, gpt_version='gpt-4o-mini', seed=42, src_lang_code='en', tgt_lang_code='ko'):
        system_prompt = self.get_gpt_system_prompt(src_lang_code, tgt_lang_code)
        
        response = self.client.chat.completions.create(
            model=gpt_version,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
            temperature=1.0,
            seed=seed,
        )
        output = response.choices[0].message.content
        return output


In [ ]:
def get_user_prompt_linebreak_modif(src_text, tgt_text, src_lang_code='en', tgt_lang_code='ko', n=1):
    src_lang_full, tgt_lang_full = LANG_TABLE[src_lang_code], LANG_TABLE[tgt_lang_code]

    task_instruction = f"""
        Two sentences will be given, a source sentence in {src_lang_full} and a target sentence in {tgt_lang_full}.
        Each sentence has a few line-breaks, but their locations are maybe not appropriate.
        There is no answer, but you can move the line-breaks' location to make the sentence more readable.

        The modification should satisfy the following conditions:
        (1) Each broken words should be semantically organized, not separated to the next line.
        (2) Each line-breaks' locations in the two sentences should be semantically/grammatically equivalent, if possible.
        (3) The number of the line-breaks can be changed, but it should not be 0 (no line-break).
        (4) If the locations are already appropriate, leave them as they are.
        (5) Never add/remove/change any words, just move the line-breaks.

        Remind the conditions, and modify the line-breaks in the sentences.
    """
    task_instruction = re.sub(r'\s{2,}', '\n', task_instruction).strip()
    task_prompt = f"<task>\n{task_instruction}\n</task>"

    output_instruction = f"""
        [SOURCE]\n{{source text with modified line-breaks}}\n[/SOURCE]
        [TARGET]\n{{target text with modified line-breaks}}\n[/TARGET]
    """
    output_instruction = re.sub(r'\s{2,}', '\n', output_instruction).strip()
    output_prompt = f"<output_template>\n{output_instruction}\n</output_template>"

    in_src_example = "I am aware that I am desperate\nfor each game and that it's\nnot natural."
    in_tgt_example = "저에게는 한 경기\n한 경기가 간절하고 당연한 게 아니라는 걸 잘\n알고 있다."
    out_src_example = "I am aware that I am desperate for each game\nand that it's not natural."
    out_tgt_example = "저에게는 한 경기 한 경기가 간절하고\n당연한 게 아니라는 걸 잘 알고 있다."

    example_instruction = f"""
        <source><English>\n{in_src_example}\n</English></source>
        <target><한국어>\n{in_tgt_example}\n</한국어></target>

        [SOURCE]\n{out_src_example}\n[/SOURCE]
        [TARGET]\n{out_tgt_example}\n[/TARGET]
    """
    example_instruction = re.sub(r'\s{2,}', '\n', example_instruction).strip()
    example_prompt = f"<example>\n{example_instruction}\n</example>"

    source_prompt = f"<source><{src_lang_full}>\n{src_text}\n</{src_lang_full}></source>"
    target_prompt = f"<target><{tgt_lang_full}>\n{tgt_text}\n</{tgt_lang_full}></target>"

    user_prompt = f"{task_prompt}\n\n{output_prompt}\n\n{example_prompt}\n\n{source_prompt}\n{target_prompt}"
    
    return user_prompt

In [ ]:
src_lang = 'en'
tgt_lang = 'ko'
src_text = "I'm hoping this email reaches\nyou today."
tgt_text = "오늘 이\n이메일이 도착하기를 바랍니다."

prompt = get_user_prompt_linebreak(src_text, tgt_text, src_lang, tgt_lang)
print(f"{prompt}")

<task>
Two sentences will be given, a source sentence in English and a target sentence in 한국어.
Each sentence has a few line-breaks, but their locations are maybe not appropriate.
There is no answer, but you can move the line-breaks' location to make the sentence more readable.
The modification should satisfy the following conditions:
(1) Each broken words should be semantically organized, not separated to the next line.
(2) Each line-breaks' locations in the two sentences should be semantically/grammatically equivalent, if possible.
(3) The number of the line-breaks can be changed, but it should not be 0 (no line-break).
(4) If the locations are already appropriate, leave them as they are.
(5) Never add/remove/change any words, just move the line-breaks.
Remind the conditions, and modify the line-breaks in the sentences.
</task>

<output_template>
[SOURCE]
{source text with modified line-breaks}
[/SOURCE]
[TARGET]
{target text with modified line-breaks}
[/TARGET]
</output_template>

<e

In [ ]:
df_lb_merged = pd.read_csv('./gpt_dpo_linebreak_modification_merged.csv')
df_lb_merged

,id,direction,in_src,in_tgt,out_src,out_tgt
0,linebreak-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...
1,linebreak-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다."
2,linebreak-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
3,linebreak-enko-4,en-ko,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...
4,linebreak-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...
...,...,...,...,...,...,...
3893,linebreak-kozh-3894,ko-zh,"주요 경제 법안인 탄력근로제 확대와 빅데이터 3법, 유치원 3법 등도 처리 여부가 ...",主要经济法案弹性劳动制规模规模扩大和大数据3法、幼儿园3法等的处理也是很灵活的。,"但是,执政党认为该方案不具备现实性,\n决定采用""250:50方案""来理顺纠结事项。","하지만 여당으로선 해당 안도 현실성이 없다고 보고,\n'250:50안'으로 가닥을 ..."
3894,linebreak-kozh-3895,ko-zh,하지만 여당으로선 해당 안도 현실성이 없다고 보고 '250:50안'으로 가닥을 잡았...,"但是,执政党认为该方案不具备现实性,决定采用""250:50方案""来理顺纠结事项。",단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。
3895,linebreak-kozh-3896,ko-zh,단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。,수고하세요\n그때 뵐게요.,辛苦了，\n到时候再见吧。
3896,linebreak-kozh-3897,ko-zh,수고하세요 그때 뵐게요.,"辛苦了,到时候再见吧。",와인 저장 장비와 와인 파이프라인을 \n정기적으로 청소하십시오.,定期对储酒设备、\n输酒管道进行清洗。


In [ ]:
bad_data_idx = df_lb_merged[df_lb_merged['in_tgt'] == '>어어!'].index[0]

df_lb_merged.loc[bad_data_idx:len(df_lb_merged)-2, 'in_src'] = df_lb_merged.loc[bad_data_idx+1:, 'in_src'].tolist()
df_lb_merged.loc[bad_data_idx:len(df_lb_merged)-2, 'in_tgt'] = df_lb_merged.loc[bad_data_idx+1:, 'in_tgt'].tolist()
df_lb_merged = df_lb_merged.iloc[:-1]

df_lb_merged

,id,direction,in_src,in_tgt,out_src,out_tgt
0,linebreak-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...
1,linebreak-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다."
2,linebreak-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
3,linebreak-enko-4,en-ko,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...
4,linebreak-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...
...,...,...,...,...,...,...
3892,linebreak-kozh-3893,ko-zh,"주요 경제 법안인 탄력근로제 확대와 빅데이터 3법, 유치원 3법 등도 처리 여부가 ...",主要经济法案弹性劳动制规模规模扩大和大数据3法、幼儿园3法等的处理也是很灵活的。,"주요 경제 법안인 탄력근로제 확대와 빅데이터 3법,\n유치원 3법 등도 처리 여부가...",主要经济法案弹性劳动制规模扩大和大数据3法、\n幼儿园3法等的处理也是很灵活的。
3893,linebreak-kozh-3894,ko-zh,하지만 여당으로선 해당 안도 현실성이 없다고 보고 '250:50안'으로 가닥을 잡았...,"但是,执政党认为该方案不具备现实性,决定采用""250:50方案""来理顺纠结事项。","但是,执政党认为该方案不具备现实性,\n决定采用""250:50方案""来理顺纠结事项。","하지만 여당으로선 해당 안도 현실성이 없다고 보고,\n'250:50안'으로 가닥을 ..."
3894,linebreak-kozh-3895,ko-zh,단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。,단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。
3895,linebreak-kozh-3896,ko-zh,수고하세요 그때 뵐게요.,"辛苦了,到时候再见吧。",수고하세요\n그때 뵐게요.,辛苦了，\n到时候再见吧。


In [ ]:
df_lb_merged.to_csv('./gpt_dpo_linebreak_modification_merged.csv', index=False)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
detector_name = 'ERCDiDip/langdetect'
model = AutoModelForSequenceClassification.from_pretrained(detector_name)
tokenizer = AutoTokenizer.from_pretrained(detector_name)

config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

2024-08-22 16:46:37.021258: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-22 16:46:37.069263: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-22 16:46:38.366007: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
detector = pipeline('text-classification', 'Manirathinam21/Multilingual_lang_detection', device='cuda')
detector

config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
text = "王妃が聞いてもいい。"
print(detector(text))

[{'label': 'Japanese', 'score': 0.9431279301643372}]


In [ ]:
import pandas as pd
from datetime import datetime
import re

In [ ]:
import re

def detect_language(text):
    patterns = {
        'en': r'[a-zA-Z]',
        'ko': r'[ㄱ-ㅎ가-힣]',
        'ja': r'[\u3040-\u309F\u30A0-\u30FF\u4e00-\u9fff]',
        'zh': r'[\u4E00-\u9FFF]',
    }

    counts = {lang: 0 for lang in patterns}
    for lang, pattern in patterns.items():
        counts[lang] = len(re.findall(pattern, text))
    total_count = sum(counts.values())
    
    if total_count == 0:
        return "No valid language detected", {}

    ratios = {lang: count / total_count for lang, count in counts.items()}
    detected_language = max(ratios, key=ratios.get)
    
    return detected_language, ratios

In [ ]:
from datetime import datetime

text_example = "In the meantime, 북 Korea의 해양 부문 연구는 연근해역에 치중되었으며 고전적 관측 장비에 의존하는 것으로 추정되어 왔다."

start_time = datetime.now()
detected_language, ratios = detect_language(text_example)
end_time = datetime.now()
elapsed_time = (end_time - start_time).total_seconds() * 1000
print(f"Detected Language: {detected_language}")
print(f"Language Ratios: {ratios}")
print(f"Elapsed Time: {elapsed_time} ms")

Detected Language: ko
Language Ratios: {'en': 0.3050847457627119, 'ko': 0.6949152542372882, 'ja': 0.0, 'zh': 0.0}
Elapsed Time: 0.087 ms


In [ ]:
text = "In the meantime, 북 Korea의 해양 부문 연구는 연근해역에 치중되었으며 고전적 관측 장비에 의존하는 것으로 추정되어 왔다."

start_time = datetime.now()
detected_language, score = detector(text)[0]
end_time = datetime.now()
elapsed_time = (end_time - start_time).total_seconds() * 1000

print(f"Detected Language: {detected_language}")
print(f"Language Score: {score}")
print(f"Elapsed Time: {elapsed_time} ms")

Detected Language: label
Language Score: score
Elapsed Time: 9.613999999999999 ms


In [ ]:
df_lb_merged = pd.read_csv('./gpt_dpo_linebreak_modification_merged.csv')

In [ ]:
df_lb_merged_filtered = df_lb_merged.copy()

trash_indices = []
for idx, row in df_lb_merged_filtered.iterrows():
    in_src_lang, _ = detect_language(row['in_src'])
    out_src_lang, _ = detect_language(row['out_src'])

    in_tgt_lang, _ = detect_language(row['in_tgt'])
    out_tgt_lang, _ = detect_language(row['out_tgt'])

    if in_src_lang != out_src_lang:
        if in_src_lang == out_tgt_lang:
            df_lb_merged_filtered.at[idx, 'out_src'], df_lb_merged_filtered.at[idx, 'out_tgt'] = df_lb_merged_filtered.at[idx, 'out_tgt'], df_lb_merged_filtered.at[idx, 'out_src']
        else:
            trash_indices.append(idx)
    else:
        if in_tgt_lang == out_tgt_lang:
            continue
        else:
            trash_indices.append(idx)

df_lb_merged_filtered.drop(index=trash_indices, inplace=True)

print(f"Before: {len(df_lb_merged)}")
print(f"After: {len(df_lb_merged_filtered)}")
display(df_lb_merged_filtered)

Before: 3897
After: 3241


,id,direction,in_src,in_tgt,out_src,out_tgt
0,linebreak-enko-1,en-ko,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...
1,linebreak-enko-2,en-ko,"However, renal biopsy confirms diagnosis for m...","그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.","However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다."
2,linebreak-enko-3,en-ko,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ..."
3,linebreak-enko-4,en-ko,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...
4,linebreak-enko-5,en-ko,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...
...,...,...,...,...,...,...
3892,linebreak-kozh-3893,ko-zh,"주요 경제 법안인 탄력근로제 확대와 빅데이터 3법, 유치원 3법 등도 처리 여부가 ...",主要经济法案弹性劳动制规模规模扩大和大数据3法、幼儿园3法等的处理也是很灵活的。,"주요 경제 법안인 탄력근로제 확대와 빅데이터 3법,\n유치원 3법 등도 처리 여부가...",主要经济法案弹性劳动制规模扩大和大数据3法、\n幼儿园3法等的处理也是很灵活的。
3893,linebreak-kozh-3894,ko-zh,하지만 여당으로선 해당 안도 현실성이 없다고 보고 '250:50안'으로 가닥을 잡았...,"但是,执政党认为该方案不具备现实性,决定采用""250:50方案""来理顺纠结事项。","하지만 여당으로선 해당 안도 현실성이 없다고 보고,\n'250:50안'으로 가닥을 ...","但是,执政党认为该方案不具备现实性,\n决定采用""250:50方案""来理顺纠结事项。"
3894,linebreak-kozh-3895,ko-zh,단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。,단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。
3895,linebreak-kozh-3896,ko-zh,수고하세요 그때 뵐게요.,"辛苦了,到时候再见吧。",수고하세요\n그때 뵐게요.,辛苦了，\n到时候再见吧。


In [ ]:
df_lb_dpo['id'] = df_lb_dpo['id'].apply(lambda x: f'dpo-{x}')
df_lb_dpo

,id,src,tgt,data_source,direction,dpo_info
0,dpo-linebreak-enko-1,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",base-1m-en,en-ko,lb-reflected
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...,base-1m-en,en-ko,lb-reflected
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
6477,dpo-linebreak-kozh-3893,"주요 경제 법안인 탄력근로제 확대와 빅데이터 3법,\n유치원 3법 등도 처리 여부가...",主要经济法案弹性劳动制规模规模扩大和大数据3法、幼儿园3法等的处理也是很灵活的。,base-1m-zh,ko-zh,lb-ignored
6478,dpo-linebreak-kozh-3894,"하지만 여당으로선 해당 안도 현실성이 없다고 보고,\n'250:50안'으로 가닥을 ...","但是,执政党认为该方案不具备现实性,决定采用""250:50方案""来理顺纠结事项。",base-1m-zh,ko-zh,lb-ignored
6479,dpo-linebreak-kozh-3895,단지 그녀가 좋아하는 일을 하고 싶어요.,我只想做她喜欢的事。,base-1m-zh,ko-zh,lb-ignored
6480,dpo-linebreak-kozh-3896,수고하세요\n그때 뵐게요.,"辛苦了,到时候再见吧。",base-1m-zh,ko-zh,lb-ignored


In [ ]:
display(df_lb_dpo['direction'].value_counts())
display(df_lb_dpo['dpo_info'].value_counts())

direction
ko-en    1300
ko-ja    1298
en-ko    1296
ko-zh    1294
ja-ko     816
zh-ko     478
Name: count, dtype: int64

dpo_info
lb-reflected    3241
lb-ignored      3241
Name: count, dtype: int64

In [ ]:
df_dpo = pd.read_csv('./dpo_unpreprocessed.csv')
df_dpo

,id,src,tgt,data_source,direction,dpo_info
0,dpo-linebreak-enko-1,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상...",base-1m-en,en-ko,lb-reflected
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...,base-1m-en,en-ko,lb-reflected
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
29714,dpo-style-koen-1296,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en,st-informal
29715,dpo-style-koen-1297,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en,st-informal
29716,dpo-style-koen-1298,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en,st-informal
29717,dpo-style-koen-1299,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en,st-informal


In [ ]:
max_lb_idx = df_dpo['dpo_info'].str.startswith('lb').idxmin()
df_dpo = df_dpo.iloc[max_lb_idx:].reset_index(drop=True)
df_dpo

,id,src,tgt,data_source,direction,dpo_info
0,dpo-propernoun-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 Angola, Kenya, Nigeria, Tanzania, Zam...",base-1m-en,en-ko,pn-maintained
1,dpo-propernoun-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,pn-maintained
2,dpo-propernoun-enko-10,"Coach Jung Jung-yong, who led the national tea...",FIFA U-20 World Cup 준우승을 차지한 U-20 대표팀 Jung Jun...,base-1m-en,en-ko,pn-maintained
3,dpo-propernoun-enko-12,"In terms of the identity of music, Western mus...","음악의 정체성 문제에 있어서, Western music는 한국의 음악으로서 정체성 ...",base-1m-en,en-ko,pn-maintained
4,dpo-propernoun-enko-26,"""Hey Hey Hey"", the love theme of Lee Soo-yeon ...",극중 주인공인 Lee Soo-yeon(Lee Je-hoon)과 Han Yeo-reu...,base-1m-en,en-ko,pn-maintained
...,...,...,...,...,...,...
21913,dpo-style-koen-1296,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en,st-informal
21914,dpo-style-koen-1297,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en,st-informal
21915,dpo-style-koen-1298,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en,st-informal
21916,dpo-style-koen-1299,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en,st-informal


In [ ]:
df_dpo = pd.concat([df_lb_dpo, df_dpo], ignore_index=True).reset_index(drop=True)
df_dpo

,id,src,tgt,data_source,direction,dpo_info
0,dpo-linebreak-enko-1,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",base-1m-en,en-ko,lb-reflected
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...,base-1m-en,en-ko,lb-reflected
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
28395,dpo-style-koen-1296,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en,st-informal
28396,dpo-style-koen-1297,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en,st-informal
28397,dpo-style-koen-1298,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en,st-informal
28398,dpo-style-koen-1299,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en,st-informal


In [ ]:
df_dpo_filtered = df_dpo.copy()

trash_indices = []
num_changed = 0
for idx, row in df_dpo_filtered.iterrows():
    if row['dpo_info'].startswith('pn'):
        continue
    
    src_lang, tgt_lang = row['direction'].split('-')
    src_lang_detected = detect_language(str(row['src']))[0]
    tgt_lang_detected = detect_language(str(row['tgt']))[0]
    if src_lang != src_lang_detected:
        if src_lang == tgt_lang_detected:
            df_dpo_filtered.at[idx, 'src'], df_dpo_filtered.at[idx, 'tgt'] = df_dpo_filtered.at[idx, 'tgt'], df_dpo_filtered.at[idx, 'src']
            num_changed += 1
        else:
            trash_indices.append(idx)
    else:
        if tgt_lang == tgt_lang_detected:
            continue
        else:
            trash_indices.append(idx)

df_dpo_filtered.drop(index=trash_indices, inplace=True)

print(f"Changed: {num_changed}")
print(f"Deleted: {len(trash_indices)}")

Changed: 6
Deleted: 7216


In [ ]:
display(df_dpo['dpo_info'].value_counts())
display(df_dpo_filtered['dpo_info'].value_counts())

dpo_info
st-formal        7681
st-informal      7681
pn-translated    3278
pn-maintained    3278
lb-ignored       3241
lb-reflected     3241
Name: count, dtype: int64

dpo_info
st-formal        5047
st-informal      4885
pn-translated    3278
pn-maintained    3278
lb-ignored       2348
lb-reflected     2348
Name: count, dtype: int64

In [ ]:
df_dpo_filtered.to_csv('./dpo_preprocessed.csv', index=False)

In [ ]:
dpo = pd.read_csv('./dpo_preprocessed.csv')
dpo

,id,src,tgt,data_source,direction,dpo_info
0,dpo-linebreak-enko-1,Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",base-1m-en,en-ko,lb-reflected
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...,base-1m-en,en-ko,lb-reflected
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
21179,dpo-style-koen-1296,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en,st-informal
21180,dpo-style-koen-1297,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en,st-informal
21181,dpo-style-koen-1298,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en,st-informal
21182,dpo-style-koen-1299,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en,st-informal


In [ ]:
guidelines = []
for idx, row in dpo.iterrows():
    dpo_info = str(row['dpo_info'])
    if dpo_info.startswith('lb'):
        suffix = '[Line-break]'
        if dpo_info.endswith('reflected'):
            guideline = "Reflect the line-breaks to the translation."
        elif dpo_info.endswith('ignored'):
            guideline = "Ignore the line-breaks to the translation."
    elif dpo_info.startswith('pn'):
        suffix = '[Proper noun]'
        if dpo_info.endswith('maintained'):
            guideline = "Maintain the proper nouns in the source language, when translating."
        elif dpo_info.endswith('translated'):
            guideline = "Translate the proper nouns to the target language, when translating."
    elif dpo_info.startswith('st'):
        suffix = '[Style]'
        if dpo_info.endswith('informal'):
            guideline = "Translate the text to formal and colloquial style."
        else:
            guideline = "Translate the text to informal and written style."
    
    guideline = f"{suffix} {guideline}"
    guidelines.append(guideline)

dpo['guideline'] = guidelines
dpo

,id,src,tgt,data_source,direction,dpo_info,guideline
0,dpo-linebreak-enko-1,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected,[Line-break] Reflect the line-breaks to the tr...
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected,[Line-break] Reflect the line-breaks to the tr...
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상...",base-1m-en,en-ko,lb-reflected,[Line-break] Reflect the line-breaks to the tr...
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...,base-1m-en,en-ko,lb-reflected,[Line-break] Reflect the line-breaks to the tr...
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected,[Line-break] Reflect the line-breaks to the tr...
...,...,...,...,...,...,...,...
29714,dpo-style-koen-1296,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en,st-informal,[Style] Translate the text to formal and collo...
29715,dpo-style-koen-1297,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en,st-informal,[Style] Translate the text to formal and collo...
29716,dpo-style-koen-1298,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en,st-informal,[Style] Translate the text to formal and collo...
29717,dpo-style-koen-1299,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en,st-informal,[Style] Translate the text to formal and collo...


In [ ]:
dpo = dpo[[
    'id', 'direction', 'dpo_info', 'guideline', 'src', 'tgt', 'data_source',
]]
dpo.to_csv('./dpo_unpreprocessed.csv', index=False)

In [ ]:
dpo = dpo[['id', 'dpo_info', 'guideline', 'src', 'tgt', 'data_source', 'direction']]
dpo

,id,dpo_info,guideline,src,tgt,data_source,direction
0,dpo-linebreak-enko-1,lb-reflected,"If line-breaks in the source sentence, REFLECT...",Blockchain is the transaction information \nth...,블록체인은 하나의 데이터베이스 장부로서 \n모든 FFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko
1,dpo-linebreak-enko-2,lb-reflected,"If line-breaks in the source sentence, REFLECT...","However, renal biopsy confirms diagnosis \nfor...","그러나, 신장 생검은 pSS 환자의 \n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko
2,dpo-linebreak-enko-3,lb-reflected,"If line-breaks in the source sentence, REFLECT...",The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",base-1m-en,en-ko
3,dpo-linebreak-enko-4,lb-reflected,"If line-breaks in the source sentence, REFLECT...",The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은\nFNS 시스템에서 필요한 정보의 제공 및 ...,base-1m-en,en-ko
4,dpo-linebreak-enko-5,lb-reflected,"If line-breaks in the source sentence, REFLECT...",On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko
...,...,...,...,...,...,...,...
21179,dpo-style-koen-1296,st-informal,Translate the source sentence in a FORMAL and ...,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en
21180,dpo-style-koen-1297,st-informal,Translate the source sentence in a FORMAL and ...,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en
21181,dpo-style-koen-1298,st-informal,Translate the source sentence in a FORMAL and ...,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en
21182,dpo-style-koen-1299,st-informal,Translate the source sentence in a FORMAL and ...,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en


In [ ]:
dpo.to_csv('./dpo_preprocessed.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

dpo_train, dpo_test = train_test_split(dpo, test_size=1000, random_state=42)
dpo_train = dpo_train.sample(n=20000, random_state=42)

print(len(dpo_train))
print(len(dpo_test))

20000
1000


In [ ]:
dpo_train = dpo_train.reset_index(drop=True)
dpo_test = dpo_test.reset_index(drop=True)
display(dpo_train.head())
display(dpo_test.head())

,id,dpo_info,guideline,src,tgt,data_source,direction
0,dpo-style-jako-3809,st-informal,Translate the source sentence in an INFORMAL a...,一部の研究者は、これが子供たちにより柔軟な言語学習者になるのに役立つと信じています。,몇몇 연구자들은 이게 애들이 더 유연한 언어 배우는 데 도움이 된다고 생각하고 있어요.,base-1m-ja,ja-ko
1,dpo-style-jako-3013,st-informal,Translate the source sentence in an INFORMAL a...,俳優のオ・ジホさんです。,여기 오지호 배우가 있어요.,base-1m-ja,ja-ko
2,dpo-propernoun-koen-3233,pn-translated,"If proper nouns in the source sentence, MUST t...",아카풀코는 과거 유명 인사들이 즐겨 찾던 세계적 휴양도시였지만 최근 들어서는 살인율...,Acapulco was a world-renowned resort city that...,base-1m-en,ko-en
3,dpo-style-koja-4191,st-formal,Translate the source sentence in a FORMAL and ...,"t+는 양이온의 운반율, z는 양이온의 전하, n은 반응에 참여한 전자 수이다.",t+は陽イオンの輸送率を、zは陽イオンの電荷を、nは反応に関与した電子の数を示しています。,base-1m-ja,ko-ja
4,dpo-linebreak-koen-1264,lb-ignored,"If line-breaks in the source sentence, IGNORE ...",타병원에서 내원 6일전부터\nDonepezil이 증량된 것을 확인하였다.,It was confirmed that the amount of donepezil ...,base-1m-en,ko-en


,id,dpo_info,guideline,src,tgt,data_source,direction
0,dpo-style-koen-96,st-formal,Translate the source sentence in a FORMAL and ...,하지만 부활절 기간 동안 희망을 버릴 사람은 없을 것 같네요.,"However, it seems unlikely that anyone would g...",base-1m-en,ko-en
1,dpo-style-jako-2634,st-formal,Translate the source sentence in a FORMAL and ...,彼が足跡を残したのはそれらの映画でした。,그가 남긴 발자취는 그 영화들이었습니다.,base-1m-ja,ja-ko
2,dpo-propernoun-jako-4676,pn-maintained,"If proper nouns in the source sentence, DO NOT...",この他にもモータースポーツ舞台に向けたFIA承認ロールケージおよびサベルトバケットシート、そ...,"이외에도 모터스포츠 무대를 위한 FIA 승인 롤-케이지 및 サベルト 버킷 시트, 그...",base-1m-ja,ja-ko
3,dpo-style-koen-547,st-informal,Translate the source sentence in an INFORMAL a...,이에 더해 각 조사별로 연령이 증가할수록 영농 활동 비중이 높아지는 추세를 보였으나...,"Also, each survey showed that as people got ol...",base-1m-en,ko-en
4,dpo-linebreak-enko-133,lb-ignored,"If line-breaks in the source sentence, IGNORE ...",The present disclosure relates to a curable tr...,"본 발명은 하나 이상의 합성 이소프렌 중합체, 스티렌계 블록 공중합체, 경화제 및 ...",base-1m-en,en-ko


In [ ]:
dpo_train.to_csv('./dpo_train.csv', index=False)
dpo_test.to_csv('./dpo_test.csv', index=False)

In [ ]:
import pandas as pd

In [ ]:
dpo = pd.read_csv('./dpo_unpreprocessed.csv')
dpo

,id,src,tgt,data_source,direction,dpo_info
0,dpo-linebreak-enko-1,Blockchain is the transaction information that...,블록체인은 하나의 데이터베이스 장부로서 모든 \nFFF를 안전하게 기록한 거래정보를...,base-1m-en,en-ko,lb-reflected
1,dpo-linebreak-enko-2,"However, renal biopsy confirms diagnosis for\n...","그러나, 신장 생검은 pSS 환자의\n최소 변화 질환에 대한 진단을 확인한다.",base-1m-en,en-ko,lb-reflected
2,dpo-linebreak-enko-3,The meeting will also be attended by first lad...,"영부인 회동에는 앙골라, 케냐, 나이지리아, \n탄자니아, 잠비아 등 아프리카 정상...",base-1m-en,en-ko,lb-reflected
3,dpo-linebreak-enko-4,The collection of neural signals by the cuff-t...,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공\n및 ...,base-1m-en,en-ko,lb-reflected
4,dpo-linebreak-enko-5,On behalf of Cafe Boulud NYC at CITY HARVEST B...,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,base-1m-en,en-ko,lb-reflected
...,...,...,...,...,...,...
29714,dpo-style-koen-1296,지난해 10월 29일 평양에서 처음으로 열린 `가을철 마라톤 애호가 경기 대회`는 ...,The 'Autumn Marathon Lover's Competition' that...,base-1m-en,ko-en,st-informal
29715,dpo-style-koen-1297,손에 몇 방울을 떨어뜨리세요.,Just put a few drops on your hand.,base-1m-en,ko-en,st-informal
29716,dpo-style-koen-1298,터치 검출 전극(TDL)은 구동 전극(COML)의 전극 패턴의 연장 방향과 직교하는...,"The touch detection electrode, or TDL, is made...",base-1m-en,ko-en,st-informal
29717,dpo-style-koen-1299,"일부 국가 덴마크, 몰타, 스위스, 포르투갈에서는 시각·청각장애인을 위한 보조적 서...","Places like Denmark, Malta, Switzerland, and P...",base-1m-en,ko-en,st-informal


In [ ]:
display(dpo['dpo_info'].value_counts())
display(dpo['direction'].value_counts())

dpo_info
st-formal        7681
st-informal      7681
lb-ignored       3900
lb-reflected     3899
pn-translated    3278
pn-maintained    3278
Name: count, dtype: int64

direction
zh-ko           5284
ko-zh           5110
ja-ko           5074
en-ko           4849
ko-ja           4808
ko-en           4592
lb-reflected       1
Name: count, dtype: int64

In [2]:
import pandas as pd

In [3]:
data_path = '../../inference/results/prime-cleansing/sft/mmt_not_prime_train.csv'
df = pd.read_csv(data_path)

In [4]:
df

,id,src,src_xcomet,tgt,tgt_xcomet,data_source,direction
0,00111-enko-0000002,"In particular, typhoons and inclement weather ...",NaN,"특히 태풍과 궂은 날씨로 일부 가을 축제가 이미 차질을 빚은데다, 지역 경제가 어려...",NaN,aihub-111,en-ko
1,00111-enko-0000003,"Currently, Korea Battery Fund No. 1 and Kolink...",NaN,현재 더블유에프엠은 한국배터리펀드1호와 코링크PE가 합산 12%의 지분을 보유한 대...,NaN,aihub-111,en-ko
2,00111-enko-0000004,"Four portraits of Lee Hang-ro, Yoo Jung-gyo, Y...",NaN,이항로·유중교·유인석·이소응 네 분의 영정은 나라를 위해 목숨을 바친 수많은 의병들...,NaN,aihub-111,en-ko
3,00111-enko-0000005,"If the veterans along with Kim Ha-sung, Lee Ju...",NaN,"베테랑과 김하성, 이정후, 김혜성 등이 시너지 효과를 일으키면 넥센은 어느 팀과 비...",NaN,aihub-111,en-ko
4,00111-enko-0000006,"Yerim village and Seopyeon village, the main v...",NaN,예림리의 본동 마을인 예림 마을과 서편 마을은 대부분 공단 지구에 편입되어 옛 흔적...,NaN,aihub-111,en-ko
...,...,...,...,...,...,...,...
2705568,71593-zhko-0035832,从分析结果可以看出，所有堆叠的破损值均小于1，说明模型所应用的复合材料均未破损。,NaN,해석 결과 모든 적층에서 파손 값이 1보다 작으므로 모델에 적용된 복합재가 파손되지...,NaN,aihub-71593,zh-ko
2705569,71593-zhko-0035833,为此，为了期待的功能性和应用，调节大小和形状的多种组成的纳米粒子的合成一直成为人们关注的对象。,NaN,이로부터 기대되는 기능성과 응용을 위하여 크기나 모양이 조절된 다양한 조성의 나노 ...,NaN,aihub-71593,zh-ko
2705570,71593-zhko-0035834,铂族金属的分离纯化主要采用湿法，几种方法结合纯化。,NaN,백금족 금속의 분리와 정제는 주로 습식법을 이용하고 있으며 몇 가지 방법을 조합하여...,NaN,aihub-71593,zh-ko
2705571,71593-zhko-0035835,添加0.1wt%的Ti纳米粒子比减少Sn-58Bi焊料的表面张力，比未添加时湿性增加了20....,NaN,0.1 wt%의 Ti 나노 입자 첨가는 Sn-58Bi 솔더의 표면 장력을 감소시켜 ...,NaN,aihub-71593,zh-ko


In [5]:
sft_en = df[df['direction'] == 'en-ko']
sft_ja = df[df['direction'] == 'ja-ko']
sft_zh = df[df['direction'] == 'zh-ko']
print(f"EN: {len(sft_en)}\nJA: {len(sft_ja)}\nZH: {len(sft_zh)}")

EN: 904218
JA: 900366
ZH: 900989


In [6]:
import sys
sys.path.append('../../inference/evaluation')
from inference_evaluation import save_line_by_line_metrics

2024-09-04 09:29:11.101833: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-04 09:29:11.152024: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 09:29:12.312461: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
save_path_en = '../../inference/results/prime-cleansing/sft/en_not_prime_train_with_xcomet.csv'
save_path_ja = '../../inference/results/prime-cleansing/sft/ja_not_prime_train_with_xcomet.csv'
save_path_zh = '../../inference/results/prime-cleansing/sft/zh_not_prime_train_with_xcomet.csv'

metric_type = 'xcomet-no-ref'
src_col, ref_col = 'src', 'tgt'

In [12]:
for save_path, df_lang in zip([save_path_en, save_path_ja, save_path_zh], [sft_en, sft_ja, sft_zh]):
    save_line_by_line_metrics(df_lang, save_path, metric_type, src_col, ref_col)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Encoder model frozen.
/home/ubuntu/miniforge3/envs/nmt/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


Calculating XCOMET-NO-REF of src...


Preparing triplets: 100%|██████████| 904218/904218 [00:32<00:00, 28117.19it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 7065/7065 [12:14:49<00:00,  6.24s/it] 
/home/ubuntu/sehyeong/tmax-enko-mt/translation_datasets/dpo/../../inference/evaluation/inference_evaluation.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df[eval_col] = xcomet_scores


Calculating XCOMET-NO-REF of tgt...


Preparing triplets: 100%|██████████| 904218/904218 [00:31<00:00, 28705.30it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0:  98%|█████████▊| 6942/7065 [10:06:44<10:45,  5.24s/it] 

In [7]:
import pandas as pd

en = pd.read_csv('../../inference/results/prime-cleansing/sft/en_not_prime_train_with_xcomet.csv')
en_sample = en.sample(n=1000, random_state=42)
en_sample

,id,src,src_xcomet-no-ref,src_xcomet,tgt,tgt_xcomet-no-ref,tgt_xcomet,data_source,direction
818828,71266-enko-0081624,Blockchain is the transaction information that...,97.218132,,블록체인은 하나의 데이터베이스 장부로서 모든 FFF를 안전하게 기록한 거래정보를 말한다.,91.668260,,aihub-71266,en-ko
209873,00124-enko-0095720,"However, renal biopsy confirms diagnosis for m...",96.726799,,"그러나, 신장 생검은 pSS 환자의 최소 변화 질환에 대한 진단을 확인한다.",73.520827,,aihub-124,en-ko
763702,71266-enko-0023212,The meeting will also be attended by first lad...,83.164227,,"영부인 회동에는 앙골라, 케냐, 나이지리아, 탄자니아, 잠비아 등 아프리카 정상의 ...",81.081975,,aihub-71266,en-ko
839657,71266-enko-0103705,The collection of neural signals by the cuff-t...,98.283398,,커프형 전극에 의한 신경 신호의 수집은 FNS 시스템에서 필요한 정보의 제공 및 피...,99.539292,,aihub-71266,en-ko
6369,00111-enko-0006865,On behalf of Cafe Boulud NYC at CITY HARVEST B...,85.969245,,지난해 10월에 있었던 유명 요리 자선행사인 'CITY HARVEST BID 201...,84.250617,,aihub-111,en-ko
...,...,...,...,...,...,...,...,...,...
680797,71265-enko-0169205,"I will process your order now, and I will send...",99.941278,,"지금 주문을 처리하고, 결제를 처리할 수 있는 링크를 보내드리겠습니다.",99.179780,,aihub-71265,en-ko
234233,00124-enko-0121519,"In this case, it may be understood that exampl...",99.722505,,"이 경우, 도 3 내지 도 17c를 참조하여 설명된 예시들이 직관적으로 적용될 수 ...",97.708702,,aihub-124,en-ko
196569,00124-enko-0081620,"Hereinafter, for convenience of description, a...",98.557043,,이하에서는 설명의 편의상 대상체에 삽입되는 의료 도구가 니들인 경우를 예로 들어 설...,88.265324,,aihub-124,en-ko
768124,71266-enko-0027873,"In the purified water control mode, the purifi...",94.689572,,정수 제어 모드에서 정수 공급 장치는 제2 정보값을 근거로 사용자에게 제공될 정수의...,88.357031,,aihub-71266,en-ko


In [2]:
import pandas as pd

In [3]:
en_sample = pd.read_csv('../../inference/results/prime-cleansing/sft/en_not_prime_dpo_sample.csv')
ja_sample = pd.read_csv('../../inference/results/prime-cleansing/sft/ja_not_prime_dpo_sample.csv')
zh_sample = pd.read_csv('../../inference/results/prime-cleansing/sft/zh_not_prime_dpo_sample.csv')

In [6]:
display(en_sample.tail(5))
display(ja_sample.tail(5))
display(zh_sample.tail(5))

,id,src,src_xcomet,tgt,tgt_xcomet,data_source,direction
32195,00563-enko-0010098,The invention relates to a technology for manu...,71.867657,그 발명품은 자동차 문패를 제조하는 기술과 관련이 있습니다. 이 기술은 (1) 도어...,80.086851,aihub-563,en-ko
32196,00563-enko-0002180,"By the present invention, in order to suppress...",76.231253,"저온 균열의 발생을 억제하기 위하여, 인장 강도가 780 ㎫ 이상인 고강도 강판을 ...",80.084074,aihub-563,en-ko
32197,00563-enko-0019378,Provided is an oxide sintered body suitably us...,82.723880,본 발명은 표시 장치용 산화물 반도체막의 제조에 적절하게 사용되는 산화물 소결체이며...,80.083799,aihub-563,en-ko
32198,00563-enko-0030659,To provide a vehicle engine start-up control d...,78.878558,엔진을 적절한 타이밍에 재시동시킬 수 있는 차량의 엔진 시동 제어 장치를 제공한다....,80.083489,aihub-563,en-ko
32199,00563-enko-0014614,The invention relates to the equipment for hig...,84.843981,본 발명은 고온고압에서 등압기술을 이용해 물질을 처리하는 장비에 관한 것이다. 액체...,80.081916,aihub-563,en-ko


,id,src,src_xcomet,tgt,tgt_xcomet,data_source,direction
32195,71411-jako-0002742,最近、初めて切り干し、大根の煮物なるものを食べました。,92.593467,"최근 처음으로 무말랭이, 무 조림을 먹었습니다.",77.398384,aihub-71411,ja-ko
32196,71411-jako-0005119,慣れた山道だったが、夕方に夕闇が降りると、ふと怖くなった。,97.618592,익숙한 산길이었지만 저녁에 거미가 깔리니 문득 두려워졌다.,77.381659,aihub-71411,ja-ko
32197,71411-jako-0000776,同作家は、今まで隠してきたこの世界の謎、全部描いていきますとも呼びかけた。,98.156929,이 작가는 지금까지 숨겨온 이 세계의 수수께끼 모두 그려나가겠다고도 입장을 표명했다.,77.380872,aihub-71411,ja-ko
32198,71411-jako-0000584,そこから、大豆から作った肉のような「大豆ミート」などが人気になっています。,93.729925,그러면서 콩으로 만든 고기 같은 '콩고기' 등이 인기를 끌고 있습니다.,77.376145,aihub-71411,ja-ko
32199,71411-jako-0004386,円錐を半分に切ったものと似た形の肺から上に狭く丸くそびえる部分を肺尖という。,88.161808,원뿔을 반 자른 것과 비슷한 모양의 허파에서 위쪽으로 좁고 둥글게 솟은 부분을 허파...,77.329469,aihub-71411,ja-ko


,id,src,src_xcomet,tgt,tgt_xcomet,data_source,direction
32092,71411-zhko-0002949,因为新冠肺炎，海外旅行受阻，本想和家人一起去济州岛，但国内旅行的费用也涨得很离谱。,79.893434,"코로나로 해외여행이 막혀 가족들과 제주도로 가려고 했는데, 국내 여행의 여행비 조차...",26.100558,aihub-71411,zh-ko
32093,71411-zhko-0000924,喂，把你认识的后台都叫来。我来叫&name&学姐和&name&学姐。,40.516692,야 있는 백 다 불러와 내가 &name& 선배랑 &name& 선배 불러올게.,25.931090,aihub-71411,zh-ko
32094,71411-zhko-0003640,不指望遇见有钱的小鲜肉，只要能遇见又高又帅的小鲜肉就很满意了，今天六点梨泰院站不见不散。,42.107949,영 앤 리치까진 아니어도 영 앤 핸썸 톨 정돈 만나보자 오늘 6시까지 이태원역 꼭.,24.750835,aihub-71411,zh-ko
32095,71411-zhko-0001735,你有加入宝妈交流贴吧吗？那里有很多信息。在你居住的地区名后面加宝妈搜索就行，你可以在那里搜索信息。,70.407230,맘카페 가입해 봤어 거기에 정보 많던데 네가 사는 곳에다가 맘 붙이면 카페 나와 거...,23.180899,aihub-71411,zh-ko
32096,71411-zhko-0000706,最强的&name&，最强的&name&，最强的&name&。,33.634305,"가장 강한 &name&, 가장 강한 &name&, 가장 강한 &name&.",21.703731,aihub-71411,zh-ko


In [7]:
import pandas as pd

In [10]:
lb = pd.read_json('./gpt_dpo_linebreak_response.jsonl', lines=True)
lb.head()

,id,custom_id,response,error
0,batch_req_06DH0Eh9YyWY32CTBBaWNcKe,linebreak-enko-1,"{'status_code': 200, 'request_id': 'ad31f66004...",NaN
1,batch_req_Kqyb74tPUwRtjSNKmIab5F2w,linebreak-enko-2,"{'status_code': 200, 'request_id': 'c07145baeb...",NaN
2,batch_req_iru8nDVE1c2lJKt92wWIwYNu,linebreak-enko-3,"{'status_code': 200, 'request_id': '0833e9068d...",NaN
3,batch_req_cqvs2mIbzGT5UELn4vD2fyHh,linebreak-enko-4,"{'status_code': 200, 'request_id': '04ef6095bf...",NaN
4,batch_req_fr4fvAA7bU9Ic1NvBcYnksBM,linebreak-enko-5,"{'status_code': 200, 'request_id': '32983cc76b...",NaN


In [18]:
lb.iloc[0]['response']

{'status_code': 200,
 'request_id': 'ad31f66004bc5023edbe9778e9abb5a2',
 'body': {'id': 'chatcmpl-A5RHRopSJj7gLnfvQ7VgfbsIzelBQ',
  'object': 'chat.completion',
  'created': 1725859869,
  'model': 'gpt-4o-mini-2024-07-18',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': "[LB_SOURCE]\nI'm going to find the treasure\nof the sea. It's a big treasure.\n[/LB_SOURCE]\n[LB_TARGET]\n바다의 보물을 찾으러\n갑니다. 큰 보물입니다.\n[/LB_TARGET]",
     'refusal': None},
    'logprobs': None,
    'finish_reason': 'stop'}],
  'usage': {'prompt_tokens': 403,
   'completion_tokens': 49,
   'total_tokens': 452},
  'system_fingerprint': 'fp_483d39d857'}}

In [22]:
from collections import defaultdict

def extract_response(df):
    data = defaultdict(list)
    for _, row in df.iterrows():
        id = row['custom_id']
        lang_pair = id.split('-')[1]
        direction = lang_pair[:2] + '-' + lang_pair[2:]

        response = row['response']['body']['choices'][0]['message']['content']

        data['id'].append(id)
        data['direction'].append(direction)
        data['response'].append(response)

    responses = pd.DataFrame(data)
    return responses

In [23]:
lb_responses = extract_response(lb)

In [24]:
lb_responses

,id,direction,response
0,linebreak-enko-1,en-ko,[LB_SOURCE]\nI'm going to find the treasure\no...
1,linebreak-enko-2,en-ko,"[LB_SOURCE]\nHowever, the temperature has incr..."
2,linebreak-enko-3,en-ko,<output_template>\n[LB_SOURCE]\nThe song was i...
3,linebreak-enko-4,en-ko,[LB_SOURCE]\nThe Facebook cover videos \nwere ...
4,linebreak-enko-5,en-ko,"[LB_SOURCE]\nAs time passes, his body\nfeels h..."
...,...,...,...
14995,linebreak-kozh-14996,ko-zh,[LB_SOURCE]\n바게트 백은 인기 스타들의\n외출 필수템이다.\n[/LB_S...
14996,linebreak-kozh-14997,ko-zh,[LB_SOURCE]\n전능 화련어가\n아니었던가?\n[/LB_SOURCE]\n[L...
14997,linebreak-kozh-14998,ko-zh,[LB_SOURCE]\n근시는 안축이 너무 길거나\n안구의 굴절력이 너무 커서\n망...
14998,linebreak-kozh-14999,ko-zh,[LB_SOURCE]\n누구든 기다리면 언제든\n엄청 벼락부자 되지 않을까?\n[/...


In [2]:
import pandas as pd

In [3]:
nm = pd.read_csv('./old/gpt_dpo_normal.csv')
lb = pd.read_csv('./old/gpt_dpo_linebreak_response_cleansed.csv')
pn = pd.read_csv('./old/gpt_dpo_propernoun_response_cleansed.csv')
st = pd.read_csv('./old/gpt_dpo_style_response_cleansed.csv')

In [4]:
print(len(nm))
print(len(lb))
print(len(pn))
print(len(st))

6497
14998
7938
6495


In [5]:
display(nm['direction'].value_counts())
display(lb['direction'].value_counts())
display(pn['direction'].value_counts())
display(st['direction'].value_counts())

direction
ko-en    1104
ja-ko    1100
ko-ja    1100
en-ko    1096
ko-zh    1049
zh-ko    1048
Name: count, dtype: int64

direction
ko-en    2504
ko-ja    2500
ko-zh    2500
ja-ko    2499
zh-ko    2499
en-ko    2496
Name: count, dtype: int64

direction
ko-zh    1600
zh-ko    1519
ko-ja    1331
ko-en    1294
ja-ko    1138
en-ko    1056
Name: count, dtype: int64

direction
ko-en    1104
ja-ko    1100
ko-ja    1098
en-ko    1096
ko-zh    1049
zh-ko    1048
Name: count, dtype: int64

In [12]:
"""
4개 데이터셋들에서 direction 별로 각각 1100개, 2500개, 1200개, 1100개씩 샘플링
만약 direction 별로 데이터가 부족한 경우, 그냥 direction 별 최대 개수만큼만 사용
"""
nm_samples = nm.groupby('direction').apply(lambda x: x.sample(n=min(1100, len(x)), random_state=42)).reset_index(drop=True)
lb_samples = lb.groupby('direction').apply(lambda x: x.sample(n=min(1100, len(x)), random_state=42)).reset_index(drop=True)
pn_samples = pn.groupby('direction').apply(lambda x: x.sample(n=min(1100, len(x)), random_state=42)).reset_index(drop=True)
st_samples = st.groupby('direction').apply(lambda x: x.sample(n=min(1100, len(x)), random_state=42)).reset_index(drop=True)

In [13]:
len(nm_samples), len(lb_samples), len(pn_samples), len(st_samples)

(6493, 6600, 6556, 6491)

In [23]:
lb_request = pd.read_json('./old/gpt_dpo_linebreak_request.jsonl', lines=True)
pn_request = pd.read_json('./old/gpt_dpo_propernoun_request.jsonl', lines=True)
st_request = pd.read_json('./old/gpt_dpo_style_request.jsonl', lines=True)

In [73]:
text = pn_request.iloc[0]['body']['messages'][1]['content']
print(text)

<instruction>
You have to generate two types of modified target text:
(1) Maintained: Only the proper nouns should be maintained in English. Other words should be translated into 한국어.
(2) Translated: The proper nouns should be translated in 한국어. Other words should also be translated into 한국어.
* Proper noun is the unique names of specific people, places, or things.
* Their could be multiple proper nouns in the text.
* If there is no proper noun in the text, the output should be "N/A" within the template.
Do not change any other parts of the text, except for the proper nouns.
Do not add XML tags or any other special characters, just follow the output template.
Use the examples as a guide and follow the instructions.
</instruction>

<output_template>
[PN_MAINTAINED]
{한국어 target text with English proper nouns, or N/A}
[/PN_MAINTAINED]
[PN_TRANSLATED]
{한국어 target text with 한국어 proper nouns, or N/A}
[/PN_TRANSLATED]
</output_template>

<example>
<example_input>
<source><English>
The Statue o

In [49]:
import re
def extract_tagged_text_for_responses(text, tag, match_num=0, square=False):
    if square:
        pattern = fr'\[{tag}\](.*?)\[/{tag}\]'
    else:
        pattern = fr'<{tag}>(.*?)<\/{tag}>'
    matches = re.findall(pattern, text, re.DOTALL)
    extracted_text = matches[match_num].strip() if matches else None
    return extracted_text

In [56]:
text_with_src_lang = extract_tagged_text_for_responses(text, 'LB_SOURCE', match_num=0, square=True)
text_with_tgt_lang = extract_tagged_text_for_responses(text, 'LB_TARGET', match_num=0, square=True)
src_lang = text_with_src_lang.split()[0][1:]
tgt_lang = text_with_tgt_lang.split()[0][1:]

src_texts = extract_tagged_text_for_responses(text, src_lang, match_num=1, square=False)
tgt_texts = extract_tagged_text_for_responses(text, tgt_lang, match_num=1, square=False)
print(src_texts)
print(tgt_texts)

I'm going to find the treasure of the sea. It's a big treasure.
바다의 보물을 찾으러 갑니다. 큰 보물입니다.


In [74]:
from tqdm import tqdm
from collections import defaultdict

LANG_TABLE_REVERSE = {
    'English': 'en',
    '한국어': 'ko',
    '日本語': 'ja',
    '中文': 'zh',
}

def extract_src_and_tgt(df):
    data = defaultdict(list)
    for _, row in tqdm(df.iterrows(), total=len(df), desc='Extracting texts'):
        text = row['body']['messages'][1]['content']
        text_with_src_lang = extract_tagged_text_for_responses(text, 'source', match_num=0, square=False)
        text_with_tgt_lang = extract_tagged_text_for_responses(text, 'target', match_num=0, square=False)
        src_lang = text_with_src_lang.split('\n')[0][1:-1]
        tgt_lang = text_with_tgt_lang.split('\n')[0][1:-1]

        src_texts = extract_tagged_text_for_responses(text, src_lang, match_num=1, square=False)
        tgt_texts = extract_tagged_text_for_responses(text, tgt_lang, match_num=1, square=False)

        data['id'].append(row['custom_id'])
        data['direction'].append(f'{LANG_TABLE_REVERSE[src_lang]}-{LANG_TABLE_REVERSE[tgt_lang]}')
        data['src'].append(src_texts)
        data['tgt'].append(tgt_texts)

    extracted_data = pd.DataFrame(data)
    return extracted_data

In [76]:
extracted_lb = extract_src_and_tgt(lb_request)
extracted_pn = extract_src_and_tgt(pn_request)
extracted_st = extract_src_and_tgt(st_request)

Extracting texts: 100%|██████████| 6497/6497 [00:00<00:00, 17823.97it/s]


In [81]:
extracted_lb.head()

,id,direction,src,tgt
0,linebreak-enko-1,en-ko,I'm going to find the treasure of the sea. It'...,바다의 보물을 찾으러 갑니다. 큰 보물입니다.
1,linebreak-enko-2,en-ko,"However, the temperature has increased by 1.2°...",그러나 피레네 기후변화관측소(OPCC)에 따르면 대산괴의 온도는 60년 동안 1.2...
2,linebreak-enko-3,en-ko,The song was included in the album of the same...,이 곡은 2012년에 동명의 앨범에 수록되었고 올가 코르무키나에 관한 다큐멘터리 영...
3,linebreak-enko-4,en-ko,The Facebook cover videos were getting viral a...,페이스북 커버 영상도 화제가 됐다.
4,linebreak-enko-5,en-ko,"As time passes, his body feels heavier and hea...",시간이 흐르면 흐를수록 몸은 점점 무거워집니다.


In [85]:
display(lb_samples.iloc[0])
display(extracted_lb[extracted_lb['id'] == lb_samples['id'][0]])

id                                         linebreak-enko-4483
direction                                                en-ko
source       Here, a pen, a touch module, \nor a microphone...
target       여기에 용도에 따라 \n펜이나 터치 모듈, \n마이크 등을 입력 장치로 \n추가할 ...
Name: 0, dtype: object

,id,direction,src,tgt
4482,linebreak-enko-4483,en-ko,"Here, a pen, a touch module, or a microphone c...","여기에 용도에 따라 펜이나 터치 모듈, 마이크 등을 입력 장치로 추가할 수 있다."


In [87]:
display(lb_samples.head(1))
display(pn_samples.head(1))
display(st_samples.head(1))

,id,direction,source,target
0,linebreak-enko-4483,en-ko,"Here, a pen, a touch module, \nor a microphone...","여기에 용도에 따라 \n펜이나 터치 모듈, \n마이크 등을 입력 장치로 \n추가할 ..."


,id,direction,maintained,translated
0,propernoun-enko-3609,en-ko,"한국은 석유수입량이 세계 4위, LNG 수입량이 세계 2위로, Azerbaijan과...","한국은 석유수입량이 세계 4위, LNG 수입량이 세계 2위로, 아제르바이잔과 같은 ..."


,id,direction,formal,informal
0,style-enko-45,en-ko,완벽한 직업은 쉽게 찾을 수 있지만 사랑은 결코 충분하지 않습니다.,완벽한 직업은 쉽게 구할 수 있지만 사랑은 절대 충분하지 않아요.


In [141]:
lb_dpo_ignored = defaultdict(list)
lb_dpo_reflected = defaultdict(list)
for idx, row in lb_samples.iterrows():
    extracted_row = extracted_lb[extracted_lb['id'] == row['id']]
    if len(extracted_row) == 0:
        print(f"ID: {row['id']} is not found in the extracted data")
        continue
    
    id_ignored = f'dpo-{row["id"]}'
    info_ignored = 'lb-ignored'
    inst_ignored = ["[Line breaks] Ignore them."]
    lb_dpo_ignored['id'].append(id_ignored)
    lb_dpo_ignored['dpo_info'].append(info_ignored)
    lb_dpo_ignored['instruction'].append(inst_ignored)
    lb_dpo_ignored['src'].append(row['source'])
    lb_dpo_ignored['tgt-chosen'].append(extracted_row['tgt'].values[0])
    lb_dpo_ignored['tgt-rejected'].append(row['target'])
    lb_dpo_ignored['direction'].append(row['direction'])

    id_reflected = f'{row["id"]}-reflected'
    info_reflected = 'lb-reflected'
    inst_reflected = ["[Line breaks] Reflect them."]
    lb_dpo_reflected['id'].append(id_reflected)
    lb_dpo_reflected['dpo_info'].append(info_reflected)
    lb_dpo_reflected['instruction'].append(inst_reflected)
    lb_dpo_reflected['src'].append(row['source'])
    lb_dpo_reflected['tgt-chosen'].append(row['target'])
    lb_dpo_reflected['tgt-rejected'].append(extracted_row['tgt'].values[0])
    lb_dpo_reflected['direction'].append(row['direction'])

lb_dpo_ignored = pd.DataFrame(lb_dpo_ignored)
lb_dpo_reflected = pd.DataFrame(lb_dpo_reflected)

In [142]:
display(lb_dpo_ignored.head())
print(len(lb_dpo_ignored))
display(lb_dpo_reflected.head())
print(len(lb_dpo_reflected))

,id,dpo_info,instruction,src,tgt-chosen,tgt-rejected,direction
0,dpo-linebreak-enko-4483,lb-ignored,[[Line breaks] Ignore them.],"Here, a pen, a touch module, \nor a microphone...","여기에 용도에 따라 펜이나 터치 모듈, 마이크 등을 입력 장치로 추가할 수 있다.","여기에 용도에 따라 \n펜이나 터치 모듈, \n마이크 등을 입력 장치로 \n추가할 ...",en-ko
1,dpo-linebreak-enko-3428,lb-ignored,[[Line breaks] Ignore them.],"Referring to FIG. 2, the driver may set \nan a...","도 2를 참조하면, 운전자는 차량에 장착된 자동 주차 모드를 설정할 수 있다.","도 2를 참조하면, 운전자는 \n차량에 장착된 \n자동 주차 모드를 설정할 수 있다.",en-ko
2,dpo-linebreak-enko-1529,lb-ignored,[[Line breaks] Ignore them.],As markets have expanded due to each country's...,각국 환경 규제 강화와 전기차 우대 정책으로 시장이 확대되면서 배터리 시장도 매년 ...,각국 환경 규제 강화와 \n전기차 우대 정책으로 시장이 \n확대되면서 배터리 시장도...,en-ko
3,dpo-linebreak-enko-4547,lb-ignored,[[Line breaks] Ignore them.],All seven newly designated \ninstitutions in 2...,2019년 신규 지정된 7개 기관이 모두 기타 공공기관으로 지정되었다.,2019년 신규 지정된 \n7개 기관이 모두 \n기타 공공기관으로 \n지정되었다.,en-ko
4,dpo-linebreak-enko-2538,lb-ignored,[[Line breaks] Ignore them.],An illustrative method according to a set of\n...,"컴퓨팅 디바이스의 메모리 상에 저장되어 있는 명령어의 세트에 따른 예시적인 방법은,...","컴퓨팅 디바이스의 메모리 상에 저장되어 있는 명령어의 세트에 따른 예시적인 방법은,...",en-ko


6600


,id,dpo_info,instruction,src,tgt-chosen,tgt-rejected,direction
0,linebreak-enko-4483-reflected,lb-reflected,[[Line breaks] Reflect them.],"Here, a pen, a touch module, \nor a microphone...","여기에 용도에 따라 \n펜이나 터치 모듈, \n마이크 등을 입력 장치로 \n추가할 ...","여기에 용도에 따라 펜이나 터치 모듈, 마이크 등을 입력 장치로 추가할 수 있다.",en-ko
1,linebreak-enko-3428-reflected,lb-reflected,[[Line breaks] Reflect them.],"Referring to FIG. 2, the driver may set \nan a...","도 2를 참조하면, 운전자는 \n차량에 장착된 \n자동 주차 모드를 설정할 수 있다.","도 2를 참조하면, 운전자는 차량에 장착된 자동 주차 모드를 설정할 수 있다.",en-ko
2,linebreak-enko-1529-reflected,lb-reflected,[[Line breaks] Reflect them.],As markets have expanded due to each country's...,각국 환경 규제 강화와 \n전기차 우대 정책으로 시장이 \n확대되면서 배터리 시장도...,각국 환경 규제 강화와 전기차 우대 정책으로 시장이 확대되면서 배터리 시장도 매년 ...,en-ko
3,linebreak-enko-4547-reflected,lb-reflected,[[Line breaks] Reflect them.],All seven newly designated \ninstitutions in 2...,2019년 신규 지정된 \n7개 기관이 모두 \n기타 공공기관으로 \n지정되었다.,2019년 신규 지정된 7개 기관이 모두 기타 공공기관으로 지정되었다.,en-ko
4,linebreak-enko-2538-reflected,lb-reflected,[[Line breaks] Reflect them.],An illustrative method according to a set of\n...,"컴퓨팅 디바이스의 메모리 상에 저장되어 있는 명령어의 세트에 따른 예시적인 방법은,...","컴퓨팅 디바이스의 메모리 상에 저장되어 있는 명령어의 세트에 따른 예시적인 방법은,...",en-ko


6600


In [134]:
pn_dpo_maintained = defaultdict(list)
pn_dpo_translated = defaultdict(list)
for idx, row in pn_samples.iterrows():
    extracted_row = extracted_pn[extracted_pn['id'] == row['id']]

    id_maintained = f'dpo-{row["id"]}'
    info_maintained = 'pn-maintained'
    inst_maintained = ["[Proper noun] Maintain them in source language."]
    pn_dpo_maintained['id'].append(id_maintained)
    pn_dpo_maintained['dpo_info'].append(info_maintained)
    pn_dpo_maintained['instruction'].append(inst_maintained)
    pn_dpo_maintained['src'].append(extracted_row['src'].values[0])
    pn_dpo_maintained['tgt-chosen'].append(row['maintained'])
    pn_dpo_maintained['tgt-rejected'].append(row['translated'])
    pn_dpo_maintained['direction'].append(row['direction'])

    id_translated = f'dpo-{row["id"]}'
    info_translated = 'pn-translated'
    inst_translated = ["[Proper noun] Translate them in target language."]
    pn_dpo_translated['id'].append(id_translated)
    pn_dpo_translated['dpo_info'].append(info_translated)
    pn_dpo_translated['instruction'].append(inst_translated)
    pn_dpo_translated['src'].append(extracted_row['src'].values[0])
    pn_dpo_translated['tgt-chosen'].append(row['translated'])
    pn_dpo_translated['tgt-rejected'].append(row['maintained'])
    pn_dpo_translated['direction'].append(row['direction'])

pn_dpo_maintained = pd.DataFrame(pn_dpo_maintained)
pn_dpo_translated = pd.DataFrame(pn_dpo_translated)

In [135]:
display(pn_dpo_maintained.head())
print(len(pn_dpo_maintained))
display(pn_dpo_translated.head())
print(len(pn_dpo_translated))

,id,dpo_info,instruction,src,tgt-chosen,tgt-rejected,direction
0,dpo-propernoun-enko-3609,pn-maintained,[[Proper noun] Maintain them in source language.],Korea is the fourth largest oil importer and t...,"한국은 석유수입량이 세계 4위, LNG 수입량이 세계 2위로, Azerbaijan과...","한국은 석유수입량이 세계 4위, LNG 수입량이 세계 2위로, 아제르바이잔과 같은 ...",en-ko
1,dpo-propernoun-enko-9498,pn-maintained,[[Proper noun] Maintain them in source language.],Foreign media outlets were also quick to repor...,한국 정부가 28일 Middle East Respiratory Syndrome (M...,한국 정부가 28일 중동호흡기증후군(메르스) 사태가 사실상 종식됐다고 선언하자 외신...,en-ko
2,dpo-propernoun-enko-9531,pn-maintained,[[Proper noun] Maintain them in source language.],Negotiations against the EU's 'Brexit' (the UK...,Negotiations against the EU의 'Brexit' (the UK ...,유럽연합의 '브렉시트(영국의 유럽연합 탈퇴)' 저지 협상이 난항을 겪고 있다.,en-ko
3,dpo-propernoun-enko-11041,pn-maintained,[[Proper noun] Maintain them in source language.],This shows that low-cost airlines have driven ...,이는 2003년 이후 EU 항공시장의 성장을 저가 항공이 견인하였음을 보여주는 것이다.,이는 2003년 이후 유럽 연합 항공시장의 성장을 저가 항공이 견인하였음을 보여주는...,en-ko
4,dpo-propernoun-enko-593,pn-maintained,[[Proper noun] Maintain them in source language.],I came here from Seoul to eat red crabs.,홍게를 먹기 위해서 Seoul에서 여기까지 왔습니다.,홍게를 먹기 위해서 서울에서 여기까지 왔습니다.,en-ko


6556


,id,dpo_info,instruction,src,tgt-chosen,tgt-rejected,direction
0,dpo-propernoun-enko-3609,pn-translated,[[Proper noun] Translate them in target langua...,Korea is the fourth largest oil importer and t...,"한국은 석유수입량이 세계 4위, LNG 수입량이 세계 2위로, 아제르바이잔과 같은 ...","한국은 석유수입량이 세계 4위, LNG 수입량이 세계 2위로, Azerbaijan과...",en-ko
1,dpo-propernoun-enko-9498,pn-translated,[[Proper noun] Translate them in target langua...,Foreign media outlets were also quick to repor...,한국 정부가 28일 중동호흡기증후군(메르스) 사태가 사실상 종식됐다고 선언하자 외신...,한국 정부가 28일 Middle East Respiratory Syndrome (M...,en-ko
2,dpo-propernoun-enko-9531,pn-translated,[[Proper noun] Translate them in target langua...,Negotiations against the EU's 'Brexit' (the UK...,유럽연합의 '브렉시트(영국의 유럽연합 탈퇴)' 저지 협상이 난항을 겪고 있다.,Negotiations against the EU의 'Brexit' (the UK ...,en-ko
3,dpo-propernoun-enko-11041,pn-translated,[[Proper noun] Translate them in target langua...,This shows that low-cost airlines have driven ...,이는 2003년 이후 유럽 연합 항공시장의 성장을 저가 항공이 견인하였음을 보여주는...,이는 2003년 이후 EU 항공시장의 성장을 저가 항공이 견인하였음을 보여주는 것이다.,en-ko
4,dpo-propernoun-enko-593,pn-translated,[[Proper noun] Translate them in target langua...,I came here from Seoul to eat red crabs.,홍게를 먹기 위해서 서울에서 여기까지 왔습니다.,홍게를 먹기 위해서 Seoul에서 여기까지 왔습니다.,en-ko


6556


In [136]:
st_dpo_formal = defaultdict(list)
st_dpo_informal = defaultdict(list)
for idx, row in st_samples.iterrows():
    extracted_row = extracted_st[extracted_st['id'] == row['id']]

    id_formal = f'dpo-{row["id"]}'
    info_formal = 'st-formal'
    inst_formal = "[Style] Formal and written style."
    st_dpo_formal['id'].append(id_formal)
    st_dpo_formal['dpo_info'].append(info_formal)
    st_dpo_formal['instruction'].append(inst_formal)
    st_dpo_formal['src'].append(extracted_row['src'].values[0])
    st_dpo_formal['tgt-chosen'].append(row['formal'])
    st_dpo_formal['tgt-rejected'].append(row['informal'])
    st_dpo_formal['direction'].append(row['direction'])

    id_informal = f'dpo-{row["id"]}'
    info_informal = 'st-informal'
    inst_informal = "[Style] Informal and colloquial style."
    st_dpo_informal['id'].append(id_informal)
    st_dpo_informal['dpo_info'].append(info_informal)
    st_dpo_informal['instruction'].append(inst_informal)
    st_dpo_informal['src'].append(extracted_row['src'].values[0])
    st_dpo_informal['tgt-chosen'].append(row['informal'])
    st_dpo_informal['tgt-rejected'].append(row['formal'])
    st_dpo_informal['direction'].append(row['direction'])

st_dpo_formal = pd.DataFrame(st_dpo_formal)
st_dpo_informal = pd.DataFrame(st_dpo_informal)

In [137]:
display(st_dpo_formal.head())
print(len(st_dpo_formal))
display(st_dpo_informal.head())
print(len(st_dpo_informal))

,id,dpo_info,instruction,src,tgt-chosen,tgt-rejected,direction
0,dpo-style-enko-45,st-formal,[Style] Formal and written style.,"You can find a perfect job so easily, but love...",완벽한 직업은 쉽게 찾을 수 있지만 사랑은 결코 충분하지 않습니다.,완벽한 직업은 쉽게 구할 수 있지만 사랑은 절대 충분하지 않아요.,en-ko
1,dpo-style-enko-1121,st-formal,[Style] Formal and written style.,The present invention relates to a pharmaceuti...,"본 발명은 당뇨 합병증, 주로 당뇨병성 신장질환의 예방 및 치료를 위한 약학 조성물...","이 발명은 당뇨 합병증, 특히 당뇨병성 신장질환을 예방하고 치료하는 약을 다루고 있...",en-ko
2,dpo-style-enko-57,st-formal,[Style] Formal and written style.,"As soon as he was appointed as the president, ...",그가 사장으로 임명되자마자 가장 먼저 한 말은 언론인은 국가에 대한 견해가 굳건해야...,그 사람이 사장 되자마자 제일 먼저 한 말이 기자는 나라에 대해 확고한 생각을 가지...,en-ko
3,dpo-style-enko-1189,st-formal,[Style] Formal and written style.,A method of the present invention that is used...,"고무 보강용 코드를 제조하기 위한 본 발명의 방법은, (i) 보강용 섬유로 유리 필...","고무 보강용 코드 만드는 방법은, (i) 유리 필라멘트가 들어간 여러 개의 제1 스...",en-ko
4,dpo-style-enko-901,st-formal,[Style] Formal and written style.,At the Supreme Court's sentencing research gro...,지난 6월 대법원 양형연구회에서는 '디지털기기를 사용한 성범죄'를 5회 이상 계속 ...,지난 6월 대법원 양형연구회에서 '디지털기기를 사용한 성범죄'를 5번 이상 계속 저...,en-ko


6491


,id,dpo_info,instruction,src,tgt-chosen,tgt-rejected,direction
0,dpo-style-enko-45,st-informal,[Style] Informal and colloquial style.,"You can find a perfect job so easily, but love...",완벽한 직업은 쉽게 구할 수 있지만 사랑은 절대 충분하지 않아요.,완벽한 직업은 쉽게 찾을 수 있지만 사랑은 결코 충분하지 않습니다.,en-ko
1,dpo-style-enko-1121,st-informal,[Style] Informal and colloquial style.,The present invention relates to a pharmaceuti...,"이 발명은 당뇨 합병증, 특히 당뇨병성 신장질환을 예방하고 치료하는 약을 다루고 있...","본 발명은 당뇨 합병증, 주로 당뇨병성 신장질환의 예방 및 치료를 위한 약학 조성물...",en-ko
2,dpo-style-enko-57,st-informal,[Style] Informal and colloquial style.,"As soon as he was appointed as the president, ...",그 사람이 사장 되자마자 제일 먼저 한 말이 기자는 나라에 대해 확고한 생각을 가지...,그가 사장으로 임명되자마자 가장 먼저 한 말은 언론인은 국가에 대한 견해가 굳건해야...,en-ko
3,dpo-style-enko-1189,st-informal,[Style] Informal and colloquial style.,A method of the present invention that is used...,"고무 보강용 코드 만드는 방법은, (i) 유리 필라멘트가 들어간 여러 개의 제1 스...","고무 보강용 코드를 제조하기 위한 본 발명의 방법은, (i) 보강용 섬유로 유리 필...",en-ko
4,dpo-style-enko-901,st-informal,[Style] Informal and colloquial style.,At the Supreme Court's sentencing research gro...,지난 6월 대법원 양형연구회에서 '디지털기기를 사용한 성범죄'를 5번 이상 계속 저...,지난 6월 대법원 양형연구회에서는 '디지털기기를 사용한 성범죄'를 5회 이상 계속 ...,en-ko


6491


In [139]:
nm_samples = nm_samples[[
    'id', 'src', 'tgt', 'direction'
]]
nm_samples['id'] = nm_samples['id'].apply(lambda x: '-'.join(['dpo']+ x.split('-')[1:]))
nm_samples['dpo_info'] = 'nm'
nm_samples['instruction'] = '["Translate plainly"]'
nm_samples.head()

,id,src,tgt,direction,dpo_info,instruction
0,dpo-enko-0023952,"You can find a perfect job so easily, but love...",완벽한 직업은 쉽게 찾을 수 있지만 사랑은 결코 그렇지 않아요.,en-ko,nm,"[""Translate plainly""]"
1,dpo-enko-0019674,The present invention relates to a pharmaceuti...,"본 발명은 당뇨 합병증, 주로 당뇨병성 신장질환의 예방 및 치료용 약학 조성물에 관...",en-ko,nm,"[""Translate plainly""]"
2,dpo-enko-0036799,"As soon as he was appointed as the president, ...","사장으로 임명되자마자 제일 처음 밝힌 것이 언론인은 국가관이 투철하고, 체제의 수호...",en-ko,nm,"[""Translate plainly""]"
3,dpo-enko-0012925,A method of the present invention that is used...,"고무 보강용 코드를 제조하기 위한 본 발명의 방법은, (i) 보강용 섬유로서 유리 ...",en-ko,nm,"[""Translate plainly""]"
4,dpo-enko-0110327,At the Supreme Court's sentencing research gro...,지난 6월 대법원 양형연구회에서는 '디지털기기를 사용한 성범죄'를 5회 이상 계속 ...,en-ko,nm,"[""Translate plainly""]"


In [143]:
total_df = pd.concat([lb_dpo_ignored, lb_dpo_reflected, pn_dpo_maintained, pn_dpo_translated, st_dpo_formal, st_dpo_informal], ignore_index=True)
total_df

,id,dpo_info,instruction,src,tgt-chosen,tgt-rejected,direction
0,dpo-linebreak-enko-4483,lb-ignored,[[Line breaks] Ignore them.],"Here, a pen, a touch module, \nor a microphone...","여기에 용도에 따라 펜이나 터치 모듈, 마이크 등을 입력 장치로 추가할 수 있다.","여기에 용도에 따라 \n펜이나 터치 모듈, \n마이크 등을 입력 장치로 \n추가할 ...",en-ko
1,dpo-linebreak-enko-3428,lb-ignored,[[Line breaks] Ignore them.],"Referring to FIG. 2, the driver may set \nan a...","도 2를 참조하면, 운전자는 차량에 장착된 자동 주차 모드를 설정할 수 있다.","도 2를 참조하면, 운전자는 \n차량에 장착된 \n자동 주차 모드를 설정할 수 있다.",en-ko
2,dpo-linebreak-enko-1529,lb-ignored,[[Line breaks] Ignore them.],As markets have expanded due to each country's...,각국 환경 규제 강화와 전기차 우대 정책으로 시장이 확대되면서 배터리 시장도 매년 ...,각국 환경 규제 강화와 \n전기차 우대 정책으로 시장이 \n확대되면서 배터리 시장도...,en-ko
3,dpo-linebreak-enko-4547,lb-ignored,[[Line breaks] Ignore them.],All seven newly designated \ninstitutions in 2...,2019년 신규 지정된 7개 기관이 모두 기타 공공기관으로 지정되었다.,2019년 신규 지정된 \n7개 기관이 모두 \n기타 공공기관으로 \n지정되었다.,en-ko
4,dpo-linebreak-enko-2538,lb-ignored,[[Line breaks] Ignore them.],An illustrative method according to a set of\n...,"컴퓨팅 디바이스의 메모리 상에 저장되어 있는 명령어의 세트에 따른 예시적인 방법은,...","컴퓨팅 디바이스의 메모리 상에 저장되어 있는 명령어의 세트에 따른 예시적인 방법은,...",en-ko
...,...,...,...,...,...,...,...
39289,dpo-style-zhko-5061,st-informal,[Style] Informal and colloquial style.,将氢经济作为一个以2030年为目标的研发项目进行推广可能会更为可取。,수소 경제를 2030년 목표로 하는 연구 개발 프로젝트로 홍보하는 게 더 좋을 거 같아.,수소 경제는 2030년을 목표로 하는 연구 개발 프로젝트로서 홍보하는 것이 더 적절...,zh-ko
39290,dpo-style-zhko-5307,st-informal,[Style] Informal and colloquial style.,我会提前准备好等您。,미리 준비해 놓고 기다릴게요.,미리 준비해 놓고 기다리겠습니다.,zh-ko
39291,dpo-style-zhko-4632,st-informal,[Style] Informal and colloquial style.,永乐帝将他封为安南国王，但他一回到家就被胡某刺杀。,"영락제가 그를 안남 왕으로 임명했는데, 집에 돌아오자마자 호씨한테 죽임을 당했어요.","영락제는 그를 안남 국왕으로 책봉했지만, 그가 귀국하자마자 호씨에게 암살당하고 말았...",zh-ko
39292,dpo-style-zhko-6435,st-informal,[Style] Informal and colloquial style.,台风总是变强，是厄尔尼诺现象吧？,태풍이 자꾸 세지는 건 엘니뇨 때문 아니야?,태풍이 자꾸 강해지는 것은 엘니뇨 현상이겠죠?,zh-ko


In [145]:
# total_df의 dpo_info-direction 조합에 따라 각각 50개씩 샘플링
test_df = total_df.groupby(['dpo_info', 'direction']).apply(lambda x: x.sample(n=min(50, len(x)), random_state=42)).reset_index(drop=True)
test_df

,id,dpo_info,instruction,src,tgt-chosen,tgt-rejected,direction
0,dpo-linebreak-enko-4519,lb-ignored,[[Line breaks] Ignore them.],There is a visual effect that\ndraws the audie...,관객의 시선이 무대로 집중되게 하는 시각적 효과가 있다.,관객의 시선이 \n무대로 집중되게 하는 \n시각적 효과가 있다.,en-ko
1,dpo-linebreak-enko-3839,lb-ignored,[[Line breaks] Ignore them.],Turkey's accession to the \nEuropean Union has...,터키의 유럽연합 가입 협상이 중단된 지 3년 만에 재개됐다.,터키의 유럽연합 가입 \n협상이 중단된 지 \n3년 만에 재개됐다.,en-ko
2,dpo-linebreak-enko-3772,lb-ignored,[[Line breaks] Ignore them.],Taiwan still has a lot of \npeople-to-people a...,대만은 여전히 중국 본토와 인적·경제적 교류가 매우 많은 편이다.,대만은 여전히 중국 본토와 \n인적·경제적 교류가 \n매우 많은 편이다.,en-ko
3,dpo-linebreak-enko-3426,lb-ignored,[[Line breaks] Ignore them.],"Nowadays, vitamin D deficiency is\ncorrelated ...",오늘날 비타민 D 결핍은 비타민 D 수용체 신호 전달에 의해 심혈관 질환과 관련이 있다.,오늘날 비타민 D 결핍은\n비타민 D 수용체 신호 전달에 의해\n심혈관 질환과 관련...,en-ko
4,dpo-linebreak-enko-3892,lb-ignored,[[Line breaks] Ignore them.],"Since the 17th, more infections have\noccurred...",칠레에서는 지난 17일부터 하루도 거르지 않고 감염자가 추가 발생하면서 보건당국을 ...,칠레에서는 지난 17일부터 하루도 거르지 않고\n감염자가 추가 발생하면서 보건당국을...,en-ko
...,...,...,...,...,...,...,...
1795,dpo-style-zhko-4473,st-informal,[Style] Informal and colloquial style.,你大可以试着接受啊？,너도 한번 해보면 어때?,당신도 시도해보면 되지 않겠습니까?,zh-ko
1796,dpo-style-zhko-5755,st-informal,[Style] Informal and colloquial style.,审核机关应当自受理申请之日起二十个工作日内完成审核工作。,심사 기관은 신청서 받은 날부터 20일 안에 심사 끝내야 해요.,심사 기관은 신청서 접수일로부터 20 영업일 이내에 심사 업무를 완료해야 합니다.,zh-ko
1797,dpo-style-zhko-4725,st-informal,[Style] Informal and colloquial style.,这源于圣经的教导，是要特别照顾自己的同族，以免同胞挨饿或受苦。,"이건 성경 가르침에서 나온 건데, 자기 동족을 잘 챙기고 배고프거나 고생하는 사람 ...","이는 성경의 가르침에서 유래한 것으로, 동족을 특별히 돌봄으로써 동포가 굶주리거나 ...",zh-ko
1798,dpo-style-zhko-4488,st-informal,[Style] Informal and colloquial style.,顺便来查查案子。,겸사겸사 사건 좀 알아보려는 거야.,겸사겸사 사건을 조사하려고 합니다.,zh-ko


In [148]:
train_df = total_df[~total_df['id'].isin(test_df['id'])].sample(frac=1, random_state=42).reset_index(drop=True)
train_df

,id,dpo_info,instruction,src,tgt-chosen,tgt-rejected,direction
0,dpo-style-koja-3732,st-formal,[Style] Formal and written style.,"1846년에 곡물법이 폐지된 뒤, 어느 한 정파가 하원에서 절대적인 우세를 확보하지...",1846年に穀物法が廃止された後、一つの派閥が下院で絶対的な優勢を確保できず、暫定的な連携で...,1846年に穀物法がなくなった後、どの派閥も下院で絶対的な力を持てなくて、短期間の協力で政治...,ko-ja
1,linebreak-koen-441-reflected,lb-reflected,[[Line breaks] Reflect them.],그래서 올해 계약이\n끝나는 것 같아요.,So I think the\ncontracts over this year.,So I think the contracts over this year.,ko-en
2,dpo-linebreak-koen-4908,lb-ignored,[[Line breaks] Ignore them.],조별활동 시 이 학생들을\n격려하여 활동에 참여하도록\n이끌어 주어야 한다.,These students should be encouraged to partici...,These students should be \nencouraged to parti...,ko-en
3,dpo-style-enko-658,st-informal,[Style] Informal and colloquial style.,"Before continuing the story, hot tea and a sim...",얘기 계속하기 전에 따뜻한 차랑 간단한 디저트가 나왔어.,이야기를 계속하기 전에 따뜻한 차와 간단한 디저트가 나왔습니다.,en-ko
4,dpo-propernoun-kozh-35973,pn-maintained,[[Proper noun] Maintain them in source language.],최근 걸그룹 화두가 굉장히 핫한 것 같아요.,最近女团体화두这个话题我觉得非常的热烈，,最近女团体这个话题我觉得非常的热烈，,ko-zh
...,...,...,...,...,...,...,...
37489,dpo-propernoun-kozh-26330,pn-maintained,[[Proper noun] Maintain them in source language.],"예컨대, 미국의 핵무기 독점은 소련 지배계급뿐 아니라 영국 지배계급에게도 도전이었다.","例如, 미국对核武器的垄断,不仅是对소련统治阶级的挑战,也是对영국统治阶级的挑战。","例如, 美国对核武器的垄断,不仅是对苏联统治阶级的挑战,也是对英国统治阶级的挑战。",ko-zh
37490,dpo-linebreak-zhko-12577,lb-ignored,[[Line breaks] Ignore them.],"据业界专家预测,\n目前约有5500家世界汽车零部件企业,\n在10年内将减少一半。","업계 전문가들은 현재 약 5,500개로 추정되는 세계 자동차부품업체들이 10년 내에...","업계 전문가들은 현재 약 5,500개로\n추정되는 세계 자동차부품업체들이,\n10년...",zh-ko
37491,linebreak-kozh-11270-reflected,lb-reflected,[[Line breaks] Reflect them.],6가지 품목 이상\n처방 비율은 병원이 \n15.3%로 가장 높았고 \n그 다음은 ...,6种以上处方比例\n最高的是医院，\n占15.3%，\n其次是诊所，\n占14.66%。,6种以上处方比例最高的是医院，占15.3%，其次是诊所，占14.66%。,ko-zh
37492,dpo-linebreak-enko-192,lb-ignored,[[Line breaks] Ignore them.],Let's find out more about which\nroad traffic ...,달라지는 도로교통법 어떤 것들이 있는지 더 자세히 알아보겠습니다.,달라지는 도로교통법 어떤 것들이\n있는지 더 자세히 알아보겠습니다.,en-ko


In [149]:
train_df.to_csv('./dpo_train.csv', index=False)
test_df.to_csv('./dpo_test.csv', index=False)

In [151]:
for idx, row in train_df.iterrows():
    if row['dpo_info'].startswith('st'):
        row['instruction'] = [row['instruction']]

for idx, row in test_df.iterrows():
    if row['dpo_info'].startswith('st'):
        row['instruction'] = [row['instruction']]

train_df.to_csv('./dpo_train.csv', index=False)
train_df.to_csv('/data/sehyeong/nmt/datasets/mmt/mmt-dpo/train.csv', index=False)
test_df.to_csv('./dpo_test.csv', index=False)
test_df.to_csv('/data/sehyeong/nmt/datasets/mmt/mmt-dpo/test.csv', index=False)